# `ROIC-Preprocessing_ver2.ipynb`

- ROIC 分析の前処理用 notebook
- 行うこと
    1. 取得したデータをデータベースに落とし込む
    2. ベンチマークのプライスとリターンデータ用意
    3. ファクター計算


In [1]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import datetime
import gc
import itertools
import logging
import os
from pathlib import Path
import sqlite3
import sys
import warnings

from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
from pandas import DataFrame
import plotly.graph_objects as go
from scipy.stats import spearmanr
import seaborn as sns
from tqdm import tqdm
import yaml

from src.analysis import returns
from src.bloomberg import data_blpapi, data_local
import src.calculate_performance_metrics as performance_metrics_utils
from src.configuration import Config, log
from src.database import sqlite_utils
from src.factset import price
import src.factset_utils as factset_utils
import src.implement_FS_BBG_formulas_utils as implement_utils
from src.preprocess import step1, step2, step3
import src.ROIC_make_data_files_ver2 as roic_utils


# import src.bloomberg_utils as bloomberg_utils

warnings.simplefilter("ignore")
config = Config.from_env()

UNIVERSE_CODE = "MSXJPN_AD"
BLOOMBERG_UNIVERSE_TICKER = "MXKO Index"

quants_dir = config.quants_dir
factset_root_dir = config.factset_root_dir
factset_financials_dir = config.factset_financials_dir
factset_index_constituents_dir = config.factset_index_constituents_dir
index_dir = factset_financials_dir / UNIVERSE_CODE
bpm_root_dir = config.bpm_root_dir
bloomberg_root_dir = config.bloomberg_root_dir
bloomberg_data_dir = config.bloomberg_data_dir
log_dir = config.log_dir


financials_db_path = index_dir / "Financials_and_Price.db"
factset_index_db_path = factset_index_constituents_dir / "Index_Constituents.db"
bloomberg_index_db_path = bloomberg_root_dir / "Index_Price_and_Returns.db"
bloomberg_valuation_db_path = bloomberg_root_dir / "Valuation.db"
bpm_db_path = bpm_root_dir / "Index_Constituents.db"

log.setup_logging(log_file="ROIC-Preprocessing_ver2.log")


In [5]:
parquet_file = (
    factset_index_constituents_dir
    / "Index_Constituents_with_Factset_code-compressed-6.parquet"
)
df = pd.read_parquet(parquet_file).sort_values("date")
print(df["date"].min(), df["date"].max())


2025-10-31 2025-10-31


In [ ]:
parquet_file = (
    factset_index_constituents_dir
    / "Index_Constituents_with_Factset_code-compressed-7.parquet"
)
df = pd.read_parquet(parquet_file).query("date>='2025-10-31'")
display(df)


,Name,Bloomberg Ticker,BloombergID,Asset ID,Asset ID Type,SEDOL,Country,GICS Sector,GICS Industry,GICS Industry Group,...,FG_COMPANY_NAME_CUSIP,P_SYMBOL_CUSIP,ISIN,FG_COMPANY_NAME_ISIN,P_SYMBOL_ISIN,CODE_JP,FG_COMPANY_NAME_CODE_JP,P_SYMBOL_CODE_JP,P_SYMBOL,FG_COMPANY_NAME
344002,ASIA CEMENT CORP,NaN,NaN,TAIAAB1,BARRAID,6056331,TWN,Materials,Construction Materials,Materials,...,アジア・セメント・コーポレーション,1102-TW,TW0001102002,アジア・セメント・コーポレーション,1102-TW,None,None,None,1102-TW,アジア・セメント・コーポレーション
344003,CAPITALAND INTEGRATED COMMERCIAL TRUST,NaN,NaN,SINBHH1,BARRAID,6420129,SGP,Real Estate,Diversified REITs,Equity Real Estate Investment Trusts (REITs),...,キャピタランド・インテグレーテッド・コマーシャル・トラスト,CPAMF-US,SG1M51904654,キャピタランド・インテグレーテッド・コマーシャル・トラスト,C38U-SG,None,None,None,C38U-SG,キャピタランド・インテグレーテッド・コマーシャル・トラスト
344004,CATHAY FINANCIAL HOLDING CO LTD,NaN,NaN,TAIAAF1,BARRAID,6425663,TWN,Financials,Insurance,Insurance,...,キャセイ・フィナンシャル・ホールディングス,2882-TW,TW0002882008,キャセイ・フィナンシャル・ホールディングス,2882-TW,None,None,None,2882-TW,キャセイ・フィナンシャル・ホールディングス
344005,CHANG HWA COMMERCIAL BANK LTD,NaN,NaN,TAIAAG1,BARRAID,6187855,TWN,Financials,Banks,Banks,...,ジャン・ホア・コマーシャル・バンク,2801-TW,TW0002801008,ジャン・ホア・コマーシャル・バンク,2801-TW,None,None,None,2801-TW,ジャン・ホア・コマーシャル・バンク
344006,KGI FINANCIAL HOLDING CO LTD,NaN,NaN,TAIAAK1,BARRAID,6431756,TWN,Financials,Insurance,Insurance,...,チャイナ・デベロップメント・フィナンシャル・ホールディング,2883-TW,TW0002883006,チャイナ・デベロップメント・フィナンシャル・ホールディング,2883-TW,None,None,None,2883-TW,チャイナ・デベロップメント・フィナンシャル・ホールディング
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2538359,BAYER AG,NaN,NaN,GERABS1,BARRAID,5069211,DEU,Health Care,Pharmaceuticals,Pharmaceuticals Biotechnology & Life Sciences,...,バイエル,BAYZF-US,DE000BAY0017,バイエル,BAYN-DE,None,None,None,BAYN-DE,バイエル
2538360,BAYERISCHE MOTOREN WERKE AG,NaN,NaN,GERABN2,BARRAID,5756030,DEU,Consumer Discretionary,Automobiles,Automobiles & Components,...,Bayerische Motoren Werke AG Pref,BYMOF-US,DE0005190037,Bayerische Motoren Werke AG Pref,BMW3-DE,None,None,None,BMW3-DE,Bayerische Motoren Werke AG Pref
2538361,BEIERSDORF AG,NaN,NaN,GERABB1,BARRAID,5107401,DEU,Consumer Staples,Personal Care Products,Household & Personal Products,...,バイヤスドルフ,BDRFF-US,DE0005200000,バイヤスドルフ,BEI-DE,None,None,None,BEI-DE,バイヤスドルフ
2538362,CONTINENTAL AG,NaN,NaN,GERACD1,BARRAID,4598589,DEU,Consumer Discretionary,Automobile Components,Automobiles & Components,...,コンチネンタル,CTTAF-US,DE0005439004,コンチネンタル,CON-DE,None,None,None,CON-DE,コンチネンタル


In [9]:
print(df["date"].unique())
df.to_parquet(
    factset_index_constituents_dir
    / "Index_Constituents_with_Factset_code-compressed-7-2.parquet",
    index=False,
)


['2025-10-31' '2025-11-30' '2025-12-31']


## 0. 現在のデータベースチェック


In [2]:
tables = sorted(sqlite_utils.get_table_names(financials_db_path))
display(tables)

with sqlite3.connect(financials_db_path) as conn:
    df_tables = pd.read_sql("SELECT * FROM FF_ASSETS", con=conn, parse_dates=["date"])
    display(df_tables)


['Active_Return_12M',
 'Active_Return_12M_annlzd',
 'Active_Return_1M',
 'Active_Return_1M_annlzd',
 'Active_Return_3M',
 'Active_Return_3M_annlzd',
 'Active_Return_3Y',
 'Active_Return_3Y_annlzd',
 'Active_Return_5Y',
 'Active_Return_5Y_annlzd',
 'Active_Return_6M',
 'Active_Return_6M_annlzd',
 'FF_ASSETS',
 'FF_ASSETS_CAGR_3Y',
 'FF_ASSETS_CAGR_3Y_PctRank',
 'FF_ASSETS_CAGR_3Y_PctRank_Sector_Neutral',
 'FF_ASSETS_CAGR_3Y_Rank',
 'FF_ASSETS_CAGR_3Y_Rank_Sector_Neutral',
 'FF_ASSETS_CAGR_3Y_ZScore',
 'FF_ASSETS_CAGR_3Y_ZScore_Sector_Neutral',
 'FF_ASSETS_CAGR_5Y',
 'FF_ASSETS_CAGR_5Y_PctRank',
 'FF_ASSETS_CAGR_5Y_PctRank_Sector_Neutral',
 'FF_ASSETS_CAGR_5Y_Rank',
 'FF_ASSETS_CAGR_5Y_Rank_Sector_Neutral',
 'FF_ASSETS_CAGR_5Y_ZScore',
 'FF_ASSETS_CAGR_5Y_ZScore_Sector_Neutral',
 'FF_ASSETS_PctRank',
 'FF_ASSETS_PctRank_Sector_Neutral',
 'FF_ASSETS_QoQ',
 'FF_ASSETS_QoQ_PctRank',
 'FF_ASSETS_QoQ_PctRank_Sector_Neutral',
 'FF_ASSETS_QoQ_Rank',
 'FF_ASSETS_QoQ_Rank_Sector_Neutral',
 'FF_AS

,date,P_SYMBOL,variable,value
0,2005-08-31,0HSW-GB,FF_ASSETS,2448.485005
1,2005-09-30,0HSW-GB,FF_ASSETS,2333.443621
2,2005-10-31,0HSW-GB,FF_ASSETS,2333.443621
3,2005-11-30,0HSW-GB,FF_ASSETS,2333.443621
4,2005-12-30,0HSW-GB,FF_ASSETS,NaN
...,...,...,...,...
791258,2025-06-30,ZURN-CH,FF_ASSETS,392546.990267
791259,2025-07-31,ZURN-CH,FF_ASSETS,392546.990267
791260,2025-08-29,ZURN-CH,FF_ASSETS,392546.990267
791261,2025-09-30,ZURN-CH,FF_ASSETS,392546.990267


## 1. BPM と Factset からダウンロードしたデータを sqlite3 に保存

- インデックス別にテーブルを作成する
- 元データは"Index_Constituents_with_Factset_code-compressed-\*.paruqet" -> 圧縮して送信した
- BPM から取得した構成比や銘柄 ID などのデータと、Factset でダウンロードした seol, cusip, isin, code_jp にそれぞれ対応する P_SYMBOL および FG_COMPANY_NAME を格納したデータ。


In [ ]:
step1.main()


## 2. Factset からダウンロードしたデータをまとめる

Financials および Price のデータをデータベースに格納する。


In [ ]:
step2.main(universe_code=UNIVERSE_CODE)


既存の 39348 行を削除し、新しいデータで上書きします。
39348 行を追加します（うち 39348 行は上書き）。
  -> FF_ASSETS: データの書き込みが完了しました。
既存の 39348 行を削除し、新しいデータで上書きします。
39348 行を追加します（うち 39348 行は上書き）。
  -> FF_BPS: データの書き込みが完了しました。
既存の 39348 行を削除し、新しいデータで上書きします。
39348 行を追加します（うち 39348 行は上書き）。
  -> FF_BPS_TANG: データの書き込みが完了しました。
既存の 39348 行を削除し、新しいデータで上書きします。
39348 行を追加します（うち 39348 行は上書き）。
  -> FF_CAPEX: データの書き込みが完了しました。
既存の 39348 行を削除し、新しいデータで上書きします。
39348 行を追加します（うち 39348 行は上書き）。
  -> FF_CASH_ST: データの書き込みが完了しました。
既存の 39348 行を削除し、新しいデータで上書きします。
39348 行を追加します（うち 39348 行は上書き）。
  -> FF_COGS: データの書き込みが完了しました。
既存の 39348 行を削除し、新しいデータで上書きします。
39348 行を追加します（うち 39348 行は上書き）。
  -> FF_COM_EQ: データの書き込みが完了しました。
既存の 39348 行を削除し、新しいデータで上書きします。
39348 行を追加します（うち 39348 行は上書き）。
  -> FF_CURR_RATIO: データの書き込みが完了しました。
既存の 39348 行を削除し、新しいデータで上書きします。
39348 行を追加します（うち 39348 行は上書き）。
  -> FF_DEBT: データの書き込みが完了しました。
既存の 39348 行を削除し、新しいデータで上書きします。
39348 行を追加します（うち 39348 行は上書き）。
  -> FF_DEBT_ENTRPR_VAL: データの書き込みが完了しました。
既存の 39348 行を削除し、新しいデータで上書きします。
39348 行を追加し

### ⚠️1AY/新規データ項目の差分更新がある場合


In [ ]:
def update_value(row, rtol=1e-5, atol=1e-3):
    """既存値をアップデートする関数"""
    existing = row["value_existing"]
    new = row["value_new"]

    # 1. 両方NaN → NaN
    if pd.isna(existing) and pd.isna(new):
        return np.nan

    # 2. 既存がNaN → 新規値
    if pd.isna(existing):
        return new

    # 3. 新規がNaN → 既存値
    if pd.isna(new):
        return existing

    # 4. 両方有効 → 数値比較
    if np.isclose(existing, new, rtol=rtol, atol=atol):
        return existing  # 同じなら既存値
    else:
        return new  # 異なれば新規値で更新


update_file = (
    INDEX_DIR / "Financials_and_Price-compressed-20241129_20251031.parquet"
)  # ファイル名は手動指定する
df_update = pd.read_parquet(update_file)
variable_list = df_update["variable"].sort_values().unique().tolist()
date_list = df_update["date"].sort_values().unique().tolist()
start_date = min(date_list)
end_date = max(date_list)

# データベースの既存テーブル
existing_tables = db_utils.get_table_names(financials_db_path)
# 新しく追加されたデータ項目があるかチェック
added_variables = list(set(variable_list) - set(existing_tables))
if len(added_variables) > 0:
    # 新規データ項目のテーブル作成
    pass

# 1AYで取得したデータを更新するテーブル一覧
update_tables = sorted(list(set(existing_tables) & set(variable_list)))
total_updated = 0
with sqlite3.connect(financials_db_path) as conn:
    for idx, table in enumerate(update_tables, 1):
        # date_listの日付のみデータベースから読み取り、df_updateとの差分を確認
        # 既存のデータが欠損していれば更新データでfillnaする（両方欠損していればそのまま）
        # 既存データと新規データの数値が異なっていれば新規データでupdateする

        print(f"\n[{idx}/{len(update_tables)}] 処理中: {table}")

        query = f"""
            SELECT
                *
            FROM
                {table}
            WHERE
                date >= '{start_date.strftime("%Y-%m-%d")}' AND date <= '{end_date.strftime("%Y-%m-%d")}'
        """
        df_existing = (
            pd.read_sql(query, con=conn, parse_dates=["date"])
            .rename(columns={"value": "value_existing"})
            .reindex(columns=["date", "P_SYMBOL", "variable", "value_existing"])
        )
        df_update_slice = (
            df_update.loc[df_update["variable"] == table]
            .copy()
            .rename(columns={"value": "value_new"})
            .reindex(columns=["date", "P_SYMBOL", "variable", "value_new"])
        )
        df_merged = pd.merge(
            df_update_slice,
            df_existing,
            on=["date", "P_SYMBOL", "variable"],
            how="outer",
        )
        df_merged["value"] = df_merged.apply(update_value, axis=1)

        # 変更検出（ベクトル化版）
        changed_mask = (
            # 既存がNaNで新規に値がある
            (df_merged["value_existing"].isna() & ~df_merged["value_new"].isna())
            |
            # 両方有効で数値が異なる
            (
                ~df_merged["value_existing"].isna()
                & ~df_merged["value_new"].isna()
                & ~np.isclose(
                    df_merged["value_existing"],
                    df_merged["value_new"],
                    rtol=1e-5,
                    atol=1e-3,
                )
            )
        )

        df_to_update = df_merged[changed_mask].reset_index(drop=True)[
            ["date", "P_SYMBOL", "variable", "value"]
        ]

        if len(df_to_update) > 0:
            print(f"  更新対象: {len(df_to_update):,}行")

            # データベース更新
            rows_affected = factset_utils.upsert_financial_data(
                df_to_update, conn, table, method="auto"
            )

            total_updated += rows_affected
        else:
            print(f"  変更なし")

print(f"\n{'=' * 50}")
print(f"📊 更新完了")
print(f"{'=' * 50}")
print(f"総更新行数: {total_updated:,}行")



[1/63] 処理中: FF_INT_EXP_NET
  更新対象: 984行
ℹ️  SQLite 3.50.4: upsert方式を使用します
❌ エラー: near "DO": syntax error


OperationalError: near "DO": syntax error

In [ ]:
update_file = (
    INDEX_DIR / "Financials_and_Price-compressed-20241129_20251031.parquet"
)  # ファイル名は手動指定する
df_update = pd.read_parquet(update_file)
variable_list = df_update["variable"].sort_values().unique().tolist()
date_list = df_update["date"].sort_values().unique().tolist()
start_date = min(date_list)
end_date = max(date_list)

# データベースの既存テーブル
existing_tables = db_utils.get_table_names(financials_db_path)
# 新しく追加されたデータ項目があるかチェック
added_variables = list(set(variable_list) - set(existing_tables))
if len(added_variables) > 0:
    # 新規データ項目のテーブル作成
    pass

# 1AYで取得したデータを更新するテーブル一覧
update_tables = list(set(existing_tables) & set(variable_list))
total_updated = 0
with sqlite3.connect(financials_db_path) as conn:
    for idx, table in enumerate(update_tables, 1):
        # date_listの日付のみデータベースから読み取り、df_updateとの差分を確認
        # 既存のデータが欠損していれば更新データでfillnaする（両方欠損していればそのまま）
        # 既存データと新規データの数値が異なっていれば新規データでupdateする

        print(f"\n[{idx}/{len(update_tables)}] 処理中: {table}")

        query = f"""
            SELECT
                *
            FROM
                {table}
            WHERE
                date >= '{start_date.strftime("%Y-%m-%d")}' AND date <= '{end_date.strftime("%Y-%m-%d")}'
        """
        df_existing = (
            pd.read_sql(query, con=conn, parse_dates=["date"])
            .rename(columns={"value": "value_existing"})
            .reindex(columns=["date", "P_SYMBOL", "variable", "value_existing"])
        )
        df_update_slice = (
            df_update.loc[df_update["variable"] == table]
            .copy()
            .rename(columns={"value": "value_new"})
            .reindex(columns=["date", "P_SYMBOL", "variable", "value_new"])
        )
        df_merged = pd.merge(
            df_update_slice,
            df_existing,
            on=["date", "P_SYMBOL", "variable"],
            how="outer",
        )
        df_merged["value"] = df_merged.apply(update_value, axis=1)

        # 変更検出（ベクトル化版）
        changed_mask = (
            # 既存がNaNで新規に値がある
            (df_merged["value_existing"].isna() & ~df_merged["value_new"].isna())
            |
            # 両方有効で数値が異なる
            (
                ~df_merged["value_existing"].isna()
                & ~df_merged["value_new"].isna()
                & ~np.isclose(
                    df_merged["value_existing"],
                    df_merged["value_new"],
                    rtol=1e-5,
                    atol=1e-3,
                )
            )
        )

        df_to_update = df_merged[changed_mask].reset_index(drop=True)[
            ["date", "P_SYMBOL", "variable", "value"]
        ]

        if len(df_to_update) > 0:
            print(f"  更新対象: {len(df_to_update):,}行")

            # データベース更新
            rows_affected = factset_utils.upsert_financial_data(
                df_to_update,
                conn,
                table,
                method="upsert",  # または "upsert"
            )

            total_updated += rows_affected
        else:
            print(f"  変更なし")

print(f"\n{'=' * 50}")
print(f"📊 更新完了")
print(f"{'=' * 50}")
print(f"総更新行数: {total_updated:,}行")



[1/63] 処理中: FF_DEBT_ST
  更新対象: 11,175行
⚠️  テーブル 'FF_DEBT_ST' にUNIQUE制約がありません
   制約を追加するには、テーブルを再作成する必要があります
⚠️  UPSERT方式を使用できません。delete_insert方式に切り替えます
  削除: 11175行, 挿入: 11175行
✅ 11175行を処理しました

[2/63] 処理中: FF_TAX_RATE
  更新対象: 10,469行
⚠️  テーブル 'FF_TAX_RATE' にUNIQUE制約がありません
   制約を追加するには、テーブルを再作成する必要があります
⚠️  UPSERT方式を使用できません。delete_insert方式に切り替えます
  削除: 10469行, 挿入: 10469行
✅ 10469行を処理しました

[3/63] 処理中: FF_ROE
  更新対象: 10,836行
⚠️  テーブル 'FF_ROE' にUNIQUE制約がありません
   制約を追加するには、テーブルを再作成する必要があります
⚠️  UPSERT方式を使用できません。delete_insert方式に切り替えます
  削除: 10836行, 挿入: 10836行
✅ 10836行を処理しました

[4/63] 処理中: FF_PPE_NET
  更新対象: 11,147行
⚠️  テーブル 'FF_PPE_NET' にUNIQUE制約がありません
   制約を追加するには、テーブルを再作成する必要があります
⚠️  UPSERT方式を使用できません。delete_insert方式に切り替えます
  削除: 11147行, 挿入: 11147行
✅ 11147行を処理しました

[5/63] 処理中: FF_SGA
  更新対象: 9,573行
⚠️  テーブル 'FF_SGA' にUNIQUE制約がありません
   制約を追加するには、テーブルを再作成する必要があります
⚠️  UPSERT方式を使用できません。delete_insert方式に切り替えます
  削除: 9573行, 挿入: 9573行
✅ 9573行を処理しました

[6/63] 処理中: FF_WKCAP
  更新対象: 11,176行
⚠️  テーブル '

In [ ]:
with sqlite3.connect(financials_db_path) as conn:
    df = pd.read_sql(
        "SELECT * FROM FF_SALES",
        con=conn,
        parse_dates=["date"],
    )
    display(df.dropna(subset=["date"]))


,date,P_SYMBOL,value,variable
0,2005-08-31,0HSW-GB,528.570465,FF_SALES
1,2005-09-30,0HSW-GB,556.819159,FF_SALES
2,2005-10-31,0HSW-GB,556.819159,FF_SALES
3,2005-11-30,0HSW-GB,556.819159,FF_SALES
4,2005-12-30,0HSW-GB,NaN,FF_SALES
...,...,...,...,...
791258,2025-06-30,ZURN-CH,37031.999926,FF_SALES
791259,2025-07-31,ZURN-CH,37031.999926,FF_SALES
791260,2025-08-29,ZURN-CH,37031.999926,FF_SALES
791261,2025-09-30,ZURN-CH,37031.999926,FF_SALES


### データベースチェック


In [2]:
# データベースの中身チェック
with sqlite3.connect(financials_db_path) as conn:
    df = pd.read_sql(
        "SELECT * FROM FF_ASSETS ORDER BY date", parse_dates=["date"], con=conn
    )
display(df)
display(df.drop_duplicates(subset=["date", "P_SYMBOL"]))
display(df["date"].unique().tolist())


,date,P_SYMBOL,variable,value
0,2005-08-31,0HSW-GB,FF_ASSETS,2448.485005
1,2005-08-31,0II3.XX1-GB,FF_ASSETS,1950.065394
2,2005-08-31,0MDJ-GB,FF_ASSETS,18381.562024
3,2005-08-31,0N1N-GB,FF_ASSETS,2893.729277
4,2005-08-31,0N3I-GB,FF_ASSETS,4176.764098
...,...,...,...,...
800327,2025-12-31,ZM-US,FF_ASSETS,11390.811000
800328,2025-12-31,ZOT-ES,FF_ASSETS,NaN
800329,2025-12-31,ZS-US,FF_ASSETS,6503.087000
800330,2025-12-31,ZTS-US,FF_ASSETS,NaN


,date,P_SYMBOL,variable,value
0,2005-08-31,0HSW-GB,FF_ASSETS,2448.485005
1,2005-08-31,0II3.XX1-GB,FF_ASSETS,1950.065394
2,2005-08-31,0MDJ-GB,FF_ASSETS,18381.562024
3,2005-08-31,0N1N-GB,FF_ASSETS,2893.729277
4,2005-08-31,0N3I-GB,FF_ASSETS,4176.764098
...,...,...,...,...
800327,2025-12-31,ZM-US,FF_ASSETS,11390.811000
800328,2025-12-31,ZOT-ES,FF_ASSETS,NaN
800329,2025-12-31,ZS-US,FF_ASSETS,6503.087000
800330,2025-12-31,ZTS-US,FF_ASSETS,NaN


[Timestamp('2005-08-31 00:00:00'),
 Timestamp('2005-09-30 00:00:00'),
 Timestamp('2005-10-31 00:00:00'),
 Timestamp('2005-11-30 00:00:00'),
 Timestamp('2005-12-30 00:00:00'),
 Timestamp('2006-01-31 00:00:00'),
 Timestamp('2006-02-28 00:00:00'),
 Timestamp('2006-03-31 00:00:00'),
 Timestamp('2006-04-28 00:00:00'),
 Timestamp('2006-05-31 00:00:00'),
 Timestamp('2006-06-30 00:00:00'),
 Timestamp('2006-07-31 00:00:00'),
 Timestamp('2006-08-31 00:00:00'),
 Timestamp('2006-09-29 00:00:00'),
 Timestamp('2006-10-31 00:00:00'),
 Timestamp('2006-11-30 00:00:00'),
 Timestamp('2006-12-29 00:00:00'),
 Timestamp('2007-01-31 00:00:00'),
 Timestamp('2007-02-28 00:00:00'),
 Timestamp('2007-03-30 00:00:00'),
 Timestamp('2007-04-30 00:00:00'),
 Timestamp('2007-05-31 00:00:00'),
 Timestamp('2007-06-29 00:00:00'),
 Timestamp('2007-07-31 00:00:00'),
 Timestamp('2007-08-31 00:00:00'),
 Timestamp('2007-09-28 00:00:00'),
 Timestamp('2007-10-31 00:00:00'),
 Timestamp('2007-11-30 00:00:00'),
 Timestamp('2007-12-

## 3. リターン&ファクター用テーブル作成


### 3-1. リターンのテーブルを作成


In [ ]:
step3.main()


問題なし


,P_SYMBOL,date,FG_PRICE,Return_1M,Forward_Return_1M,Return_1M_annlzd,Forward_Return_1M_annlzd,Return_3M,Forward_Return_3M,Return_3M_annlzd,...,Return_12M_annlzd,Forward_Return_12M_annlzd,Return_3Y,Forward_Return_3Y,Return_3Y_annlzd,Forward_Return_3Y_annlzd,Return_5Y,Forward_Return_5Y,Return_5Y_annlzd,Forward_Return_5Y_annlzd
0,0HSW-GB,2005-08-31,3.225569,NaN,0.127660,NaN,1.531916,NaN,0.342199,NaN,...,NaN,0.547014,NaN,0.847736,NaN,0.282579,NaN,0.847736,NaN,0.169547
1,0II3.XX1-GB,2005-08-31,4.166256,NaN,0.014796,NaN,0.177557,NaN,0.060419,NaN,...,NaN,-0.117139,NaN,0.115485,NaN,0.038495,NaN,0.115485,NaN,0.023097
2,0MDJ-GB,2005-08-31,6.092351,NaN,0.045704,NaN,0.548446,NaN,0.015539,NaN,...,NaN,0.021938,NaN,0.036546,NaN,0.012182,NaN,0.416530,NaN,0.083306
3,0N1N-GB,2005-08-31,5.655000,NaN,0.038904,NaN,0.466844,NaN,0.015031,NaN,...,NaN,0.052166,NaN,0.404067,NaN,0.134689,NaN,0.366932,NaN,0.073386
4,0N3I-GB,2005-08-31,2.810000,NaN,0.028470,NaN,0.341637,NaN,0.025801,NaN,...,NaN,0.012456,NaN,-0.470641,NaN,-0.156880,NaN,0.153025,NaN,0.030605


既存の 0 行との重複をチェックしました。811195 行を新たに追加します。
  -> Return_1M: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。811195 行を新たに追加します。
  -> Forward_Return_1M: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。811195 行を新たに追加します。
  -> Return_1M_annlzd: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。811195 行を新たに追加します。
  -> Forward_Return_1M_annlzd: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。811195 行を新たに追加します。
  -> Return_3M: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。811195 行を新たに追加します。
  -> Forward_Return_3M: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。811195 行を新たに追加します。
  -> Return_3M_annlzd: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。811195 行を新たに追加します。
  -> Forward_Return_3M_annlzd: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。811195 行を新たに追加します。
  -> Return_6M: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。811195 行を新たに追加します。
  -> Forward_Return_6M: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。811195 行を新たに追加します。
  -> Return_6M_annlzd: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。811195 行を新たに追加します。
  -> Forward_Return_6M_annlzd: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。811195 行を新たに追

### 3-2. インデックスの価格とリターンデータを取得 via Blpapi

⚠️ 注意 ⚠️ Bloomberg Terminal を起動している必要あり。


In [2]:
# データベース確認
with sqlite3.connect(bloomberg_index_db_path) as conn:
    df = pd.read_sql("SELECT * FROM PX_LAST", con=conn, parse_dates=["Date"])
display(df)
print(
    f"Date: {df['Date'].min().strftime('%Y-%m-%d')} 〜 {df['Date'].max().strftime('%Y-%m-%d')} ({len(df['Date'].unique()):,}日)"
)
print(f"Ticker: {df['Ticker'].nunique():,}銘柄\n\t{df['Ticker'].unique().tolist()}")


,Date,Ticker,value,variable
0,2000-01-03,MXKO Index,1356.99,PX_LAST
1,2000-01-04,MXKO Index,1312.02,PX_LAST
2,2000-01-05,MXKO Index,1300.44,PX_LAST
3,2000-01-06,MXKO Index,1290.97,PX_LAST
4,2000-01-07,MXKO Index,1326.68,PX_LAST
...,...,...,...,...
372995,2025-11-24,SPX Index,6705.12,PX_LAST
372996,2025-11-25,SPX Index,6765.88,PX_LAST
372997,2025-11-26,SPX Index,6812.61,PX_LAST
372998,2025-11-28,SPX Index,6849.09,PX_LAST


Date: 2000-01-03 〜 2025-12-01 (6,761日)
Ticker: 57銘柄
	['MXKO Index', 'MXKO0CD Index', 'MXKO0CS Index', 'MXKO0EN Index', 'MXKO0FN Index', 'MXKO0HC Index', 'MXKO0IN Index', 'MXKO0IT Index', 'MXKO0MT Index', 'MXKO0UT Index', 'MXWD Index', 'MXWD0CD Index', 'MXWD0CS Index', 'MXWD0EN Index', 'MXWD0FN Index', 'MXWD0HC Index', 'MXWD0IN Index', 'MXWD0IT Index', 'MXWD0MT Index', 'MXWD0RE Index', 'MXWD0ST Index', 'MXWD0UT Index', 'MXWDJ Index', 'S5AUCO Index', 'S5BANKX Index', 'S5CODU Index', 'S5COMS Index', 'S5COND Index', 'S5CONS Index', 'S5CPGS Index', 'S5DIVF Index', 'S5ENRS Index', 'S5ENRSX Index', 'S5FDBT Index', 'S5FDSR Index', 'S5FINL Index', 'S5HCES Index', 'S5HLTH Index', 'S5HOTR Index', 'S5HOUS Index', 'S5INDU Index', 'S5INFT Index', 'S5INSU Index', 'S5MATR Index', 'S5MATRX Index', 'S5MEDA Index', 'S5PHRM Index', 'S5RETL Index', 'S5RLST Index', 'S5SFTW Index', 'S5TECH Index', 'S5TELS Index', 'S5TELSX Index', 'S5TRAN Index', 'S5UTIL Index', 'S5UTILX Index', 'SPX Index']


In [ ]:
bbg_ticker = data_local.BloombergTickers()

tickers_kokusai = bbg_ticker.get_msci_kokusai_tickers()
display(tickers_kokusai)


['MXKO Index',
 'MXKO0CS Index',
 'MXKO0CD Index',
 'MXKO0ST Index',
 'MXKO0EN Index',
 'MXKO0FN Index',
 'MXKO0HC Index',
 'MXKO0IN Index',
 'MXKO0IT Index',
 'MXKO0MT Index',
 'MXKO0RE Index',
 'MXKO0UT Index']

In [ ]:
# yamlで設定した銘柄リストの読み込み（Bloomberg Ticker）
BLOOMBERG_TICKER_YAML = bloomberg_root_dir / "ticker-description.yaml"
EQUITY_TYPES = {"equity_index", "equity_sector_index", "equity_industry_index"}

with open(BLOOMBERG_TICKER_YAML, encoding="utf-8") as f:
    ticker_descriptions = yaml.safe_load(f)

tickers_to_download = [
    ticker["bloomberg_ticker"]
    for ticker in ticker_descriptions
    if (ticker.get("type") in EQUITY_TYPES)
    & (not ticker.get("bloomberg_ticker").startswith("MXWDJ"))
]
display(tickers_to_download)


['SPX Index',
 'S5TELS Index',
 'S5COND Index',
 'S5CONS Index',
 'S5ENRS Index',
 'S5FINL Index',
 'S5HLTH Index',
 'S5INDU Index',
 'S5INFT Index',
 'S5MATR Index',
 'S5RLST Index',
 'S5UTIL Index',
 'S5AUCO Index',
 'S5BANKX Index',
 'S5CPGS Index',
 'S5COMS Index',
 'S5RETL Index',
 'S5CODU Index',
 'S5HOTR Index',
 'S5FDSR Index',
 'S5ENRSX Index',
 'S5DIVF Index',
 'S5FDBT Index',
 'S5HCES Index',
 'S5HOUS Index',
 'S5INSU Index',
 'S5MATRX Index',
 'S5MEDA Index',
 'S5PHRM Index',
 'S5SFTW Index',
 'S5TECH Index',
 'S5TELSX Index',
 'S5TRAN Index',
 'S5UTILX Index',
 'MXKO Index',
 'MXWD Index',
 'MXWD0CS Index',
 'MXWD0CD Index',
 'MXWD0ST Index',
 'MXWD0EN Index',
 'MXWD0FN Index',
 'MXWD0HC Index',
 'MXWD0IN Index',
 'MXWD0IT Index',
 'MXWD0MT Index',
 'MXWD0RE Index',
 'MXWD0UT Index',
 'MXKO0CS Index',
 'MXKO0CD Index',
 'MXKO0ST Index',
 'MXKO0EN Index',
 'MXKO0FN Index',
 'MXKO0HC Index',
 'MXKO0IN Index',
 'MXKO0IT Index',
 'MXKO0MT Index',
 'MXKO0RE Index',
 'MXKO0UT In

In [ ]:
# -----------------------------------------------------
# Bloombergから価格データをダウンロードしてデータベースを更新
# -----------------------------------------------------


blp = data_blpapi.BlpapiFetcher()
data = blp.get_historical_data(
    securities=tickers_to_download,
    id_type="ticker",
    fields=["BEST_PE_RATIO"],
    start_date="20000101",
    end_date=datetime.datetime.today().strftime("%Y%m%d"),
)

display(data)


Bloombergセッションを開始しています...
セッション開始成功。
✅ サービスオープン完了。リクエスト作成中...
📡 リクエストを送信します [DAILY]
   期間: 2000-01-01 - 2026-01-30

✅ データ取得完了。接続を終了しました。

📊 取得データ:
   行数: 380,136行
   日付範囲: 2000-01-03 00:00:00 ~ 2026-01-30 00:00:00
   ユニーク日数: 6805日
   識別子数: 58
   識別子タイプ: TICKER
   周期: DAILY


Ticker,MXKO Index,MXKO0CD Index,MXKO0CS Index,MXKO0EN Index,MXKO0FN Index,MXKO0HC Index,MXKO0IN Index,MXKO0IT Index,MXKO0MT Index,MXKO0RE Index,...,S5RETL Index,S5RLST Index,S5SFTW Index,S5TECH Index,S5TELS Index,S5TELSX Index,S5TRAN Index,S5UTIL Index,S5UTILX Index,SPX Index
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,26.8719,33.7406,22.2253,35.2418,20.0766,30.4059,28.2768,64.7712,22.6140,NaN,...,41.5221,NaN,79.6441,58.0849,28.1268,28.1268,14.4584,13.9751,13.9751,29.3081
2000-01-04,25.9814,32.6202,21.8230,34.4944,19.5019,29.4186,27.5634,61.3260,22.4299,NaN,...,40.0147,NaN,75.4437,54.8075,26.8821,26.8821,14.0146,13.9912,13.9912,28.1840
2000-01-05,25.7520,32.2397,21.7877,35.0393,19.3196,29.6445,27.4777,59.8713,22.7539,NaN,...,39.5095,NaN,73.6348,54.6032,27.2532,27.2532,14.0103,14.5119,14.5119,28.2377
2000-01-06,25.5645,32.1248,22.1680,35.9415,19.5178,30.1938,27.8251,56.7456,23.2656,NaN,...,39.7759,NaN,70.2581,52.6117,26.5656,26.5656,14.6815,14.7144,14.7144,28.2653
2000-01-07,26.2717,33.0847,22.8795,36.2333,19.7972,31.9613,28.5851,58.9665,23.5701,NaN,...,41.9346,NaN,72.5712,52.9230,26.5338,26.5338,14.8136,14.9660,14.9660,28.9965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-01-26,23.6337,28.7319,21.1523,16.3845,15.5113,20.5285,28.0071,35.6779,23.8022,23.8350,...,26.5984,39.0535,33.3748,31.5184,23.3782,10.8888,16.7445,19.6923,19.6923,26.0468
2026-01-27,23.7374,28.5631,21.2737,16.5620,15.5368,20.3770,27.7759,36.2292,23.9318,23.8252,...,26.9896,39.0630,33.6795,32.0656,23.3810,10.7080,16.6000,20.4042,20.4042,26.1080
2026-01-28,23.8344,28.4318,21.1551,16.7336,15.4937,20.2097,27.4272,36.5178,24.1169,23.8537,...,26.7240,38.7974,33.8506,31.9856,24.8877,10.9440,16.3948,20.3561,20.3561,26.3215


In [13]:
# -----------------------------------------------------
# Bloombergから価格データをダウンロードしてデータベースを更新
# -----------------------------------------------------


# blp = bloomberg_utils.BlpapiCustom()

# 新規ティッカーがある場合

# df = blp.get_historical_data(
#     securities=tickers_to_download,
#     fields=["PX_LAST"],
#     start_date="20000101",
#     end_date=datetime.datetime.today().strftime("%Y%m%d"),
# )
# df = pd.melt(
#     df.reset_index(), id_vars=["Date"], var_name="Ticker", value_name="value"
# ).assign(variable="PX_LAST")
# display(df)
# blp.store_to_database(
#     df=df,
#     db_path=bloomberg_index_db_path,
#     table_name="PX_LAST",
#     primary_keys=["Date", "Ticker", "variable"],
#     verbose=True,
# )

# 　既存データの更新

rows_updated = blp.update_historical_data(
    db_path=bloomberg_index_db_path,
    table_name="PX_LAST",
    tickers=tickers_to_download,
    id_type="ticker",
    field="PX_LAST",
    default_start_date=datetime.datetime(2000, 1, 1),
    verbose=True,
)
print(f"\n{'=' * 60}")
print(f"処理完了: {rows_updated:,}行を処理しました")
print(f"{'=' * 60}")


📊 増分更新モード
   最新データ日付: 2025-12-01
   取得期間: 2025-12-02 ~ 2026-01-30
   対象銘柄: 58銘柄
Bloombergセッションを開始しています...
セッション開始成功。
✅ サービスオープン完了。リクエスト作成中...
📡 リクエストを送信します [DAILY]
   期間: 2025-12-02 - 2026-01-30

✅ データ取得完了。接続を終了しました。

📊 取得データ:
   行数: 2,263行
   日付範囲: 2025-12-02 00:00:00 ~ 2026-01-29 00:00:00
   ユニーク日数: 43日
   識別子数: 55
   識別子タイプ: TICKER
   周期: DAILY

📈 取得データ:
   行数: 2,263行
   日付範囲: 2025-12-02 00:00:00 ~ 2026-01-29 00:00:00
   ユニーク日数: 43日

❌ エラー発生: 'BlpapiFetcher' object has no attribute 'store_to_database'

処理完了: 0行を処理しました


Traceback (most recent call last):
  File "c:\Users\hatay_bb\Projects\Quants\src\bloomberg\data_fetch.py", line 1568, in update_historical_data
    rows_saved = self.store_to_database(
                 ^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'BlpapiFetcher' object has no attribute 'store_to_database'


In [ ]:
step3.store_index_price()


📊 増分更新モード
   最新データ日付: 2025-12-01
   取得期間: 2025-12-02 ~ 2026-01-30
   対象銘柄: 58銘柄
Bloombergセッションを開始しています...
セッション開始成功。
✅ サービスオープン完了。リクエスト作成中...
📡 リクエストを送信します [DAILY]
   期間: 2025-12-02 - 2026-01-30


Traceback (most recent call last):
  File "c:\Users\hatay_bb\Projects\Quants\src\bloomberg\data_blpapi.py", line 1571, in update_historical_data
    rows_saved = self.store_to_database(
                 ^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'BlpapiFetcher' object has no attribute 'store_to_database'
[2026-01-30 16:45:37,674 | store_index_price | INFO] - 
[2026-01-30 16:45:37,676 | store_index_price | INFO] - 処理完了: 0行を処理しました
[2026-01-30 16:45:37,677 | store_index_price | INFO] - ============================================================



✅ データ取得完了。接続を終了しました。

📊 取得データ:
   行数: 2,263行
   日付範囲: 2025-12-02 00:00:00 ~ 2026-01-29 00:00:00
   ユニーク日数: 43日
   識別子数: 55
   識別子タイプ: TICKER
   周期: DAILY

📈 取得データ:
   行数: 2,263行
   日付範囲: 2025-12-02 00:00:00 ~ 2026-01-29 00:00:00
   ユニーク日数: 43日

❌ エラー発生: 'BlpapiFetcher' object has no attribute 'store_to_database'


In [ ]:
# 既存のFG_PRICEのテーブルからリターン計算すべき日付を取得
df_index_price_filtered = (
    sqlite_utils.get_rows_by_unique_values(
        source_db_path=financials_db_path,
        target_db_path=bloomberg_index_db_path,
        source_table="FG_PRICE",
        target_table="PX_LAST",
        source_column="date",
        target_column="Date",
    )
    .query(f"Ticker == '{BLOOMBERG_UNIVERSE_TICKER}'")
    .reset_index(drop=True)
    .assign(Date=lambda x: pd.to_datetime(x["Date"]))
)


# インデックスについて同様にリターンを計算してデータベースに保存
df_index_return = returns.calculate_return_multi_periods(
    df_price=df_index_price_filtered,
    date_column="Date",
    symbol_column="Ticker",
    price_column="value",
)
print("--- return data ---")
display(df_index_return.tail(3))
print("-" * 20)

# ------------------------------------------------------------------------------------
# データチェック
# 銘柄によってはdateが1カ月ずつ連続でデータがあるとは限らない
# 価格データがない場合にpct_changeを素直に実行するとリターンの期間が他の銘柄とずれる
# そのため、全dateの長さと銘柄ごとのdateの長さを比較する
# ------------------------------------------------------------------------------------

df_check = df_index_return.reset_index()
symbol_date_counts = df_check.groupby("Ticker")["Date"].nunique()
all_date_len = len(df_check["Date"].unique())
not_enough_len_symbols = symbol_date_counts[symbol_date_counts != all_date_len].index
if len(not_enough_len_symbols) > 0:
    print("問題あり")
    display(not_enough_len_symbols)
else:  # 問題なければデータベースに保存
    print("問題なし")
    df_index_return.reset_index(inplace=True)
    display(df_index_return.tail(5))
    for col in [
        s
        for s in df_index_return.columns
        if s.startswith("Return") or s.startswith("Forward_Return")
    ]:
        df_slice = (
            df_index_return[["Date", "Ticker", col]]
            .rename(columns={col: "value"})
            .assign(variable=col)
        )
        df_slice["value"] = df_slice["value"].astype(float)
        df_slice["Date"] = pd.to_datetime(df_slice["Date"])
        sqlite_utils.delete_table_from_database(
            db_path=bloomberg_index_db_path, table_name=col
        )
        blp.store_to_database(
            df=df_slice,
            db_path=bloomberg_index_db_path,
            table_name=col,
            primary_keys=["Date", "Ticker", "variable"],
        )


--- return data ---


value variable  Return_1M  Forward_Return_1M  \
Ticker     Date                                                         
MXKO Index 2025-08-29  4339.70  PX_LAST   0.022468           0.031714   
           2025-09-30  4477.33  PX_LAST   0.031714           0.018609   
           2025-10-31  4560.65  PX_LAST   0.018609                NaN   

                       Return_1M_annlzd  Forward_Return_1M_annlzd  Return_3M  \
Ticker     Date                                                                
MXKO Index 2025-08-29          0.269611                  0.380570   0.081942   
           2025-09-30          0.380570                  0.223312   0.069486   
           2025-10-31          0.223312                       NaN   0.074525   

                       Forward_Return_3M  Return_3M_annlzd  \
Ticker     Date                                              
MXKO Index 2025-08-29                NaN          0.327766   
           2025-09-30                NaN          0.277946   
           2025-10-31                NaN          0.298101   

                       Forward_Return_3M_annlzd  ...  Return_12M_annlzd  \
Ticker     Date                                  ...                      
MXKO Index 2025-08-29                       NaN  ...           0.143251   
           2025-09-30                       NaN  ...           0.157818   
           2025-10-31                       NaN  ...           0.202551   

                       Forward_Return_12M_annlzd  Return_3Y  \
Ticker     Date                                               
MXKO Index 2025-08-29                        NaN   0.598328   
           2025-09-30                        NaN   0.819180   
           2025-10-31                        NaN   0.725760   

                       Forward_Return_3Y  Return_3Y_annlzd  \
Ticker     Date                                              
MXKO Index 2025-08-29                NaN          0.199443   
           2025-09-30                NaN          0.273060   
           2025-10-31                NaN          0.241920   

                       Forward_Return_3Y_annlzd  Return_5Y  Forward_Return_5Y  \
Ticker     Date                                                                 
MXKO Index 2025-08-29                       NaN   0.726859                NaN   
           2025-09-30                       NaN   0.854051                NaN   
           2025-10-31                       NaN   0.952408                NaN   

                       Return_5Y_annlzd  Forward_Return_5Y_annlzd  
Ticker     Date                                                    
MXKO Index 2025-08-29          0.145372                       NaN  
           2025-09-30          0.170810                       NaN  
           2025-10-31          0.190482                       NaN  

[3 rows x 26 columns]

--------------------
問題なし


,Ticker,Date,value,variable,Return_1M,Forward_Return_1M,Return_1M_annlzd,Forward_Return_1M_annlzd,Return_3M,Forward_Return_3M,...,Return_12M_annlzd,Forward_Return_12M_annlzd,Return_3Y,Forward_Return_3Y,Return_3Y_annlzd,Forward_Return_3Y_annlzd,Return_5Y,Forward_Return_5Y,Return_5Y_annlzd,Forward_Return_5Y_annlzd
238,MXKO Index,2025-06-30,4186.43,PX_LAST,0.043729,0.013833,0.524753,0.165993,0.109511,0.069486,...,0.148449,NaN,0.591242,NaN,0.197081,NaN,0.865951,NaN,0.173190,NaN
239,MXKO Index,2025-07-31,4244.34,PX_LAST,0.013833,0.022468,0.165993,0.269611,0.119433,0.074525,...,0.147621,NaN,0.493686,NaN,0.164562,NaN,0.797764,NaN,0.159553,NaN
240,MXKO Index,2025-08-29,4339.70,PX_LAST,0.022468,0.031714,0.269611,0.380570,0.081942,NaN,...,0.143251,NaN,0.598328,NaN,0.199443,NaN,0.726859,NaN,0.145372,NaN
241,MXKO Index,2025-09-30,4477.33,PX_LAST,0.031714,0.018609,0.380570,0.223312,0.069486,NaN,...,0.157818,NaN,0.819180,NaN,0.273060,NaN,0.854051,NaN,0.170810,NaN
242,MXKO Index,2025-10-31,4560.65,PX_LAST,0.018609,NaN,0.223312,NaN,0.074525,NaN,...,0.202551,NaN,0.725760,NaN,0.241920,NaN,0.952408,NaN,0.190482,NaN


✅ 保存完了。テーブル 'Return_1M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_1M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_1M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_1M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_3M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_3M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_3M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_3M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_6M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_6M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_6M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_6M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_12M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_12M' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_12M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_12M_annlzd' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Return_3Y' に 243 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'Forward_Return_3Y' に 243 行を処理し

### 3-3. Active Return 計算

テーブル作成し、データベースに保存


In [ ]:
return_cols = [
    "Return_1M",
    "Return_1M_annlzd",
    "Forward_Return_1M",
    "Forward_Return_1M_annlzd",
    "Return_3M",
    "Return_3M_annlzd",
    "Forward_Return_3M",
    "Forward_Return_3M_annlzd",
    "Return_6M",
    "Return_6M_annlzd",
    "Forward_Return_6M",
    "Forward_Return_6M_annlzd",
    "Return_12M",
    "Return_12M_annlzd",
    "Forward_Return_12M",
    "Forward_Return_12M_annlzd",
    "Return_3Y",
    "Return_3Y_annlzd",
    "Forward_Return_3Y",
    "Forward_Return_3Y_annlzd",
    "Return_5Y",
    "Return_5Y_annlzd",
    "Forward_Return_5Y",
    "Forward_Return_5Y_annlzd",
]

# ------------------------------------
# インデックスのリターン
# ------------------------------------
union_queries_index = []
for table in return_cols:
    union_queries_index.append(
        f"SELECT Date, Ticker, value, variable FROM '{table}' WHERE Ticker = '{BLOOMBERG_UNIVERSE_TICKER}'"
    )
union_query_index = " UNION ALL ".join(union_queries_index)

# データ取得
with sqlite3.connect(bloomberg_index_db_path) as conn:
    df_return_index = pd.read_sql(
        union_query_index, con=conn, parse_dates=["Date"]
    ).rename(columns={"Date": "date", "Ticker": "symbol"})

df_return_index = df_return_index.drop_duplicates(ignore_index=True)
print(f"✅ インデックスデータ: {len(df_return_index):,}件")


# ------------------------------------
# 個別銘柄のリターン
# ------------------------------------
union_queries_security = []
for table in return_cols:
    union_queries_security.append(
        f"SELECT date, P_SYMBOL, value, variable FROM '{table}'"
    )

union_query_security = " UNION ALL ".join(union_queries_security)

# データ取得
with sqlite3.connect(financials_db_path) as conn:
    df_return_security = pd.read_sql(
        union_query_security, con=conn, parse_dates=["date"]
    ).rename(columns={"P_SYMBOL": "symbol"})

df_return_security = df_return_security.drop_duplicates(ignore_index=True)

print(f"✅ 個別銘柄データ: {len(df_return_security):,}件")

# ------------------------------------
# concatenate returns
# ------------------------------------
df_returns = pd.concat([df_return_index, df_return_security], ignore_index=True)
display(df_returns.tail(3))

# ------------------------------------
# アクティブリターン計算
# ------------------------------------
df_active_returns = performance_metrics_utils.calculate_active_returns_vectorized(
    df_returns=df_returns,
    return_cols=return_cols,
    benchmark_ticker=BLOOMBERG_UNIVERSE_TICKER,
    verbose=False,
)
display(df_active_returns.tail(3))

# ------------------------------------
# Active return: データベース保存
# ------------------------------------
# 推奨方法：直列書き込み版
results = factset_utils.insert_active_returns_optimized_sqlite(
    df_active_returns=df_active_returns,
    return_cols=return_cols,
    db_path=financials_db_path,
    benchmark_ticker=BLOOMBERG_UNIVERSE_TICKER,
    batch_size=10000,
    verbose=True,
)


✅ インデックスデータ: 5,832件
✅ 個別銘柄データ: 19,076,472件


,date,symbol,value,variable
19082301,2025-10-31,ZS-US,NaN,Forward_Return_5Y_annlzd
19082302,2025-10-31,ZTS-US,NaN,Forward_Return_5Y_annlzd
19082303,2025-10-31,ZURN-CH,NaN,Forward_Return_5Y_annlzd


,date,symbol,value,variable
19076469,2025-08-29,ZURN-CH,NaN,Forward_Active_Return_5Y_annlzd
19076470,2025-09-30,ZURN-CH,NaN,Forward_Active_Return_5Y_annlzd
19076471,2025-10-31,ZURN-CH,NaN,Forward_Active_Return_5Y_annlzd


⚡ アクティブリターン最適化バッチ保存（SQLite機能活用版）
   処理列数: 24列
   データ行数: 19,076,472行
   バッチサイズ (executemany): 10,000行
⏳ データ前処理中...
✅ 前処理完了 (69.37秒)
   処理対象: 24テーブル


💾 保存中:   4%|▍         | 1/24 [00:09<03:36,  9.43s/it]

...

✅ Active_Return_5Y_annlzd: 794,853件（挿入試行）


💾 保存中:  96%|█████████▌| 23/24 [03:57<00:10, 10.44s/it]

✅ Forward_Active_Return_5Y: 794,853件（挿入試行）


💾 保存中: 100%|██████████| 24/24 [04:10<00:00, 10.42s/it]

✅ Forward_Active_Return_5Y_annlzd: 794,853件（挿入試行）
📊 バッチ保存完了統計
   成功: 24/24テーブル
   失敗: 0テーブル
   総試行件数: 19,076,472件
   前処理時間: 69.37秒
   保存時間: 250.06秒
   合計時間: 319.43秒
   スループット (試行件数ベース): 59,720件/秒
   成功率: 100.0%


### 🧪PE 取得テスト from Bloomberg（⚠️ データ取得は難しそう）

- Forward PE(BEST_PE_RATIO), Trailing PE(PE_RATIO), と Forward EPS(BEST_EPS), Trailing EPS(TRAIL_12M_EPS_BEF_XO_ITEM)を取得

- (- Forward PE と Trailing PE の差分のカラムを追加
- PEG ratio = BEST_EPS / BEST_PE_RATIO のカラムを追加
  )


In [ ]:
fields = ["BEST_PE_RATIO", "BEST_EPS", "PE_RATIO", "TRAIL_12M_EPS_BEF_XO_ITEM"]
# --------------------------------------------------------
# sedolとdateのリストを取得
# --------------------------------------------------------
with sqlite3.connect(factset_index_db_path) as conn:
    sedol_list = pd.read_sql(f"SELECT DISTINCT `SEDOL` FROM {UNIVERSE_CODE}", con=conn)[
        "SEDOL"
    ].tolist()
    sedol_list = [s + " Equity" for s in sedol_list]
    date_list = pd.read_sql(
        f"SELECT DISTINCT `date` FROM {UNIVERSE_CODE} ORDER BY `date`",
        con=conn,
        parse_dates=["date"],
    )["date"].tolist()
print("SEDOLと日付のリスト取得完了")

blp = bloomberg_utils.BlpapiCustom()
for field in fields:
    df = (
        blp.get_historical_data_with_overrides(
            securities=sedol_list,
            id_type="sedol",
            fields=[field],
            start_date=min(date_list).strftime("%Y%m%d"),
            end_date=max(date_list).strftime("%Y%m%d"),
            periodicity="MONTHLY",
            # overrides={"BEST_FPERIOD_OVERRIDE": "1FQ"},   # デフォルトで1FQ
            verbose=True,
        )
        .sort_values(["Date", "Identifier"], ignore_index=True)
        .drop(columns=["ID_Type"])
        .assign(Date=lambda x: pd.to_datetime(x["Date"]))
    )

    df = (
        pd.melt(
            df,
            id_vars=["Date", "Identifier"],
            value_vars=[field],
            var_name="variable",
        )
        .rename(columns={"Identifier": "SEDOL"})
        .assign(SEDOL=lambda x: x["SEDOL"].str.replace(" Equity", ""))
    )

    # blp.store_to_database(
    #     df=df,
    #     db_path=bloomberg_valuation_db_path,
    #     table_name=field,
    #     primary_keys=["Date", "SEDOL", "variable"],
    # )


SEDOLと日付のリスト取得完了
Bloombergセッションを開始しています...
セッション開始成功。
✅ サービスオープン完了。リクエスト作成中...
📡 リクエストを送信します[SEDOL] [MONTHLY]
   期間: 2000-01-31 - 2025-10-31

✅ データ取得完了。接続を終了しました。

📊 取得データ:
   行数: 21行
   日付範囲: 2023-11-30 00:00:00 ~ 2025-07-31 00:00:00
   ユニーク日数: 21日
   識別子数: 1
   識別子タイプ: SEDOL
   周期: MONTHLY
Bloombergセッションを開始しています...
セッション開始成功。
✅ サービスオープン完了。リクエスト作成中...
📡 リクエストを送信します[SEDOL] [MONTHLY]
   期間: 2000-01-31 - 2025-10-31

✅ データ取得完了。接続を終了しました。

📊 取得データ:
   行数: 13行
   日付範囲: 2023-11-30 00:00:00 ~ 2025-09-30 00:00:00
   ユニーク日数: 13日
   識別子数: 1
   識別子タイプ: SEDOL
   周期: MONTHLY
Bloombergセッションを開始しています...
セッション開始成功。
✅ サービスオープン完了。リクエスト作成中...
📡 リクエストを送信します[SEDOL] [MONTHLY]
   期間: 2000-01-31 - 2025-10-31

✅ データ取得完了。接続を終了しました。

📊 取得データ:
   行数: 305行
   日付範囲: 2000-01-31 00:00:00 ~ 2025-10-31 00:00:00
   ユニーク日数: 305日
   識別子数: 1
   識別子タイプ: SEDOL
   周期: MONTHLY
Bloombergセッションを開始しています...
セッション開始成功。
✅ サービスオープン完了。リクエスト作成中...
📡 リクエストを送信します[SEDOL] [MONTHLY]
   期間: 2000-01-31 - 2025-10-31

✅ データ取得完了。接続を終了しました。

📊 取得デー

✅ データベースチェック


In [ ]:
with sqlite3.connect(bloomberg_valuation_db_path) as conn:
    df = pd.read_sql("SELECT * FROM BEST_PE_RATIO", con=conn, parse_dates=["Date"])
    display(df["Date"].sort_values().unique())
    display(df.sort_values("Date", ignore_index=True).drop_duplicates())


<DatetimeArray>
['2000-01-31 00:00:00', '2000-02-29 00:00:00', '2000-03-31 00:00:00',
 '2000-04-28 00:00:00', '2000-05-31 00:00:00', '2000-06-30 00:00:00',
 '2000-07-31 00:00:00', '2000-08-31 00:00:00', '2000-09-29 00:00:00',
 '2000-10-31 00:00:00',
 ...
 '2025-04-30 00:00:00', '2025-05-29 00:00:00', '2025-05-30 00:00:00',
 '2025-06-30 00:00:00', '2025-07-31 00:00:00', '2025-08-29 00:00:00',
 '2025-08-31 00:00:00', '2025-09-30 00:00:00', '2025-10-30 00:00:00',
 '2025-10-31 00:00:00']
Length: 334, dtype: datetime64[ns]

,Date,SEDOL,variable,value
0,2000-01-31,2206301,BEST_PE_RATIO,11.744
1,2000-01-31,2213981,BEST_PE_RATIO,10.710
2,2000-02-29,2206301,BEST_PE_RATIO,13.396
3,2000-02-29,2213981,BEST_PE_RATIO,10.454
4,2000-03-31,2206301,BEST_PE_RATIO,14.650
...,...,...,...,...
61076,2025-10-31,2852533,BEST_PE_RATIO,26.723
61077,2025-10-31,2849472,BEST_PE_RATIO,13.087
61078,2025-10-31,2842255,BEST_PE_RATIO,33.487
61079,2025-10-31,2838555,BEST_PE_RATIO,22.003


✅ 欠損割合チェック


In [ ]:
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)

# bloomberg per
with sqlite3.connect(bloomberg_valuation_db_path) as conn:
    df_forward_pe = pd.read_sql(
        "SELECT * FROM BEST_PE_RATIO", parse_dates=["Date"], con=conn
    ).rename(columns={"value": "BEST_PE_RATIO", "Date": "date"})
    df_actual_pe = pd.read_sql(
        "SELECT * FROM PE_RATIO", parse_dates=["Date"], con=conn
    ).rename(columns={"value": "PE_RATIO", "Date": "date"})
    df_actual_pe.drop(columns=["variable"], inplace=True)
    df_pe = pd.merge(
        df_forward_pe, df_actual_pe, on=["date", "SEDOL"], how="outer"
    ).assign(date=lambda x: x["date"] + pd.offsets.MonthEnd(0))

df_merged = pd.merge(df_weight, df_pe, on=["date", "SEDOL"], how="outer").dropna(
    subset=["Weight (%)", "BEST_PE_RATIO", "PE_RATIO"], how="any", ignore_index=True
)

display(df_merged.tail(5))


,date,P_SYMBOL,SEDOL,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),variable,BEST_PE_RATIO,PE_RATIO
39444,2025-10-31,SBAC-US,BZ6TS23,SBAコミュニケーションズ Class A,Real Estate,Equity Real Estate Investment Trusts (REITs),0.026502,BEST_PE_RATIO,22.317,20.7390
39445,2025-10-31,CHTR-US,BZ6VT82,チャーター・コミュニケーションズ Class A,Communication Services,Media & Entertainment,0.026749,BEST_PE_RATIO,5.755,5.8559
39446,2025-10-31,WTC-AU,BZ8GX83,ワイズテック・グローバル,Information Technology,Software & Services,0.011702,BEST_PE_RATIO,57.203,74.8462
39447,2025-10-31,VST-US,BZ8VJQ8,ビストラ,Utilities,Utilities,0.082303,BEST_PE_RATIO,21.327,39.5135
39448,2025-10-31,BCP-PT,BZCNN35,Banco Comercial Portugues S.A.,Financials,Banks,0.009457,BEST_PE_RATIO,10.628,11.8584


In [ ]:
# 欠損
g = pd.DataFrame(df_merged.groupby(["date"])["Weight (%)"].agg("sum")).reset_index()
display(g.query("date>='2023-01-01'"))


,date,Weight (%)
271,2023-01-31,54.865295
272,2023-02-28,58.768044
273,2023-03-31,61.187943
274,2023-04-30,61.089016
275,2023-05-31,62.986537
276,2023-06-30,63.706548
277,2023-07-31,64.796422
278,2023-08-31,66.780949
279,2023-09-30,67.820945
280,2023-10-31,67.305035


#### Bloomberg valuation データについてメモ

- 直近 1,2 年分程度しか forward+actual pe は取れない


## 4. Factor 計算


### 4-0. Define Utility Functions


In [ ]:
def load_factor_and_weight(descriptor_list: list[str]) -> pd.DataFrame:
    # -----------------------------------
    # load data
    # -----------------------------------

    df_factor = factset_utils.load_financial_data(
        financials_db_path=financials_db_path, factor_list=descriptor_list
    )

    df_weight = factset_utils.load_index_constituents(
        factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
    )
    df = (
        factset_utils.merge_idx_constituents_and_financials(
            df_weight=df_weight, df_factor=df_factor
        )
        .fillna(np.nan)
        .dropna(subset=descriptor_list, how="all")
        .dropna(subset=["Weight (%)"], ignore_index=True)
    )

    return df


### 4-1. 成長率計算（Factset）


#### 1. QoQ, YoY, 3Yr CAGR, 5Yr CAGR の値を計算し、データベースに保存する。


In [ ]:
factor_list = [
    "FF_SALES",
    "FF_EBITDA_OPER",
    "FF_EBIT_OPER",
    "FF_EPS",
    "FF_OPER_CF",
    "FF_ASSETS",
    "FF_COM_EQ",
    "FF_DEBT",
    "FF_DEBT_LT",
    "FF_DEBT_ST",
    "FF_OPER_INC",
    "FF_CAPEX",
    "FF_FREE_CF",
    "FF_EPS_DIL",
]
period_list = ["QoQ", "YoY", "CAGR_3Y", "CAGR_5Y"]


In [ ]:
query = [f"SELECT * FROM `{table}`" for table in factor_list]
query = " UNION ALL ".join(query)

# ------------------------------------------------------
# load from database
# ------------------------------------------------------
with sqlite3.connect(financials_db_path) as conn:
    df_all = (
        pd.read_sql(query, con=conn, parse_dates=["date"])
        .sort_values("date", ignore_index=True)
        .astype({"variable": "category", "P_SYMBOL": "category"})
    ).sort_values(["variable", "P_SYMBOL", "date"], ignore_index=True)

display(df_all)
# groupby("variable")を使うことで、df_all全体を何度も走査(loc)するコストをゼロにする
# observed=True はcategory型を使う場合の高速化オプション
grouped = df_all.groupby("variable", observed=True)

# ------------------------------------------------------
# calculate growth and store to database
# ------------------------------------------------------
total_steps = len(factor_list)
for factor_name, df_factor in tqdm(grouped, total=total_steps, desc="Factors"):
    # df_factorはView(参照)の可能性があるため、計算用にコピーを作成
    # ここでメモリを食うが、factor単位なので全体コピーよりは軽い
    # かつ、ループの最後で解放される
    df_base = df_factor.copy()

    for growth in period_list:
        new_variable_name = f"{factor_name}_{growth}"
        df_result = roic_utils.calculate_growth(
            df=df_base, data_name=str(factor_name), growth_type=growth
        )

        # store to database
        db_utils.delete_table_from_database(
            db_path=financials_db_path, table_name=new_variable_name
        )
        factset_utils.store_to_database(
            df=df_result,
            db_path=financials_db_path,
            table_name=new_variable_name,
            verbose=False,
        )

    # ---- メモリ管理 ----
    # 一つのファクター処理が終わったら、使用した一時変数を削除してGC実行
    del df_base
    gc.collect()


,date,P_SYMBOL,value,variable
0,2005-08-31,0HSW-GB,2448.485005,FF_ASSETS
1,2005-09-30,0HSW-GB,2333.443621,FF_ASSETS
2,2005-10-31,0HSW-GB,2333.443621,FF_ASSETS
3,2005-11-30,0HSW-GB,2333.443621,FF_ASSETS
4,2005-12-30,0HSW-GB,NaN,FF_ASSETS
...,...,...,...,...
11077677,2025-06-30,ZURN-CH,37031.999926,FF_SALES
11077678,2025-07-31,ZURN-CH,37031.999926,FF_SALES
11077679,2025-08-29,ZURN-CH,37031.999926,FF_SALES
11077680,2025-09-30,ZURN-CH,37031.999926,FF_SALES


Factors: 100%|██████████| 14/14 [05:52<00:00, 25.15s/it]


#### 2. ファクターのランクを計算し、データベースに保存する。


- 成長率計算のため、winsorize 処理あり
- ランク inversed = False


In [ ]:
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))
factset_utils.process_rank_calculation_store_to_db(
    df_weight=df_weight,
    factor_list=factor_list,
    financials_db_path=financials_db_path,
    period_list=period_list,
)


🚀 処理開始: 56 件のタスク (Multi-Period Mode)


Rank計算進捗: 100%|██████████| 56/56 [43:08<00:00, 46.22s/it]  


🎉 全てのランク計算・保存が完了しました


In [ ]:
table_names = db_utils.get_table_names(financials_db_path)
display([s for s in table_names if ("QoQ" in s) or ("YoY" in s) or ("CAGR" in s)])

with sqlite3.connect(financials_db_path) as conn:
    df = pd.read_sql(
        "SELECT * FROM FF_OPER_INC_QoQ_PctRank ORDER BY 'date'",
        con=conn,
        parse_dates=["date"],
    )
    display(df.head())


['FF_EBITDA_OPER_QoQ',
 'FF_EBITDA_OPER_YoY',
 'FF_EBITDA_OPER_CAGR_3Y',
 'FF_EBITDA_OPER_CAGR_5Y',
 'FF_EBIT_OPER_QoQ',
 'FF_EBIT_OPER_YoY',
 'FF_EBIT_OPER_CAGR_3Y',
 'FF_EBIT_OPER_CAGR_5Y',
 'FF_EPS_QoQ',
 'FF_EPS_YoY',
 'FF_EPS_CAGR_3Y',
 'FF_EPS_CAGR_5Y',
 'FF_OPER_CF_QoQ',
 'FF_OPER_CF_YoY',
 'FF_OPER_CF_CAGR_3Y',
 'FF_OPER_CF_CAGR_5Y',
 'FF_DEBT_QoQ',
 'FF_DEBT_YoY',
 'FF_DEBT_CAGR_3Y',
 'FF_DEBT_CAGR_5Y',
 'FF_DEBT_LT_QoQ',
 'FF_DEBT_LT_YoY',
 'FF_DEBT_LT_CAGR_3Y',
 'FF_DEBT_LT_CAGR_5Y',
 'FF_DEBT_ST_QoQ',
 'FF_DEBT_ST_YoY',
 'FF_DEBT_ST_CAGR_3Y',
 'FF_DEBT_ST_CAGR_5Y',
 'FF_EBITDA_OPER_QoQ_Rank',
 'FF_EBITDA_OPER_QoQ_PctRank',
 'FF_EBITDA_OPER_QoQ_ZScore',
 'FF_EBITDA_OPER_YoY_Rank',
 'FF_EBITDA_OPER_YoY_PctRank',
 'FF_EBITDA_OPER_YoY_ZScore',
 'FF_EBITDA_OPER_CAGR_3Y_Rank',
 'FF_EBITDA_OPER_CAGR_3Y_PctRank',
 'FF_EBITDA_OPER_CAGR_3Y_ZScore',
 'FF_EBITDA_OPER_CAGR_5Y_Rank',
 'FF_EBITDA_OPER_CAGR_5Y_PctRank',
 'FF_EBITDA_OPER_CAGR_5Y_ZScore',
 'FF_EBIT_OPER_QoQ_Rank',
 'FF_EBIT_

,date,P_SYMBOL,variable,value
0,2005-11-30,0HSW-GB,FF_OPER_INC_QoQ_PctRank,0.066176
1,2005-11-30,0II3.XX1-GB,FF_OPER_INC_QoQ_PctRank,0.784810
2,2005-11-30,0MDJ-GB,FF_OPER_INC_QoQ_PctRank,0.426966
3,2005-11-30,0N1N-GB,FF_OPER_INC_QoQ_PctRank,0.475124
4,2005-11-30,0N3I-GB,FF_OPER_INC_QoQ_PctRank,0.343284


### 4-2. 成長加速度（🚧 工事中）


#### 1. YoY - 3Yr CAGR を計算、データベースに保存する


In [ ]:
descriptor_list = [
    "FF_SALES_YoY",
    "FF_SALES_CAGR_3Y",
    "FF_OPER_INC_YoY",
    "FF_OPER_INC_CAGR_3Y",
]


In [ ]:
query = [f"SELECT * FROM `{table}`" for table in descriptor_list]
query = " UNION ALL ".join(query)

# ------------------------------------------------------
# load from database
# ------------------------------------------------------
with sqlite3.connect(financials_db_path) as conn:
    df_all = (
        pd.read_sql(query, con=conn, parse_dates=["date"])
        .sort_values("date", ignore_index=True)
        .astype({"variable": "category", "P_SYMBOL": "category"})
    ).sort_values(["variable", "P_SYMBOL", "date"], ignore_index=True)

display(df_all)
df_all_acc, stats = roic_utils.calc_relative_absolute_acceleration(
    df=df_all, descriptor_list=["FF_SALES", "FF_OPER_INC"]
)
display(df_all_acc)
# groupby("variable")を使うことで、df_all全体を何度も走査(loc)するコストをゼロにする
# observed=True はcategory型を使う場合の高速化オプション
# grouped = df_all.groupby("variable", observed=True)

# ------------------------------------------------------
# calculate growth and store to database
# ------------------------------------------------------
# total_steps = len(descriptor_list)
# for factor_name, df_factor in tqdm(grouped, total=total_steps, desc="Factors"):

#     # df_factorはView(参照)の可能性があるため、計算用にコピーを作成
#     # ここでメモリを食うが、factor単位なので全体コピーよりは軽い
#     # かつ、ループの最後で解放される
#     df_base = df_factor.copy()

#     for growth in period_list:
#         new_variable_name = f"{factor_name}_{growth}"
#         df_result = roic_utils.calculate_growth(
#             df=df_base, data_name=str(factor_name), growth_type=growth
#         )

#         # store to database
#         db_utils.delete_table_from_database(
#             db_path=financials_db_path, table_name=new_variable_name
#         )
#         factset_utils.store_to_database(
#             df=df_result,
#             db_path=financials_db_path,
#             table_name=new_variable_name,
#             verbose=False,
#         )

# ---- メモリ管理 ----
# 一つのファクター処理が終わったら、使用した一時変数を削除してGC実行
# del df_base
# gc.collect()


,date,P_SYMBOL,variable,value
0,2008-08-29,0MDJ-GB,FF_OPER_INC_CAGR_3Y,-0.151382
1,2008-09-30,0MDJ-GB,FF_OPER_INC_CAGR_3Y,-0.151382
2,2008-10-31,0MDJ-GB,FF_OPER_INC_CAGR_3Y,-0.151382
3,2008-11-28,0MDJ-GB,FF_OPER_INC_CAGR_3Y,-0.151382
4,2008-12-31,0MDJ-GB,FF_OPER_INC_CAGR_3Y,-0.200381
...,...,...,...,...
2112217,2025-06-30,ZURN-CH,FF_SALES_YoY,-0.145586
2112218,2025-07-31,ZURN-CH,FF_SALES_YoY,-0.145586
2112219,2025-08-29,ZURN-CH,FF_SALES_YoY,-0.145586
2112220,2025-09-30,ZURN-CH,FF_SALES_YoY,-0.145586


🧹 自動クリーニング実行中（groupby=date）...
データクリーニング統計（グルーピング: date）

FF_SALES_YoY:
  Total: 656,034
  Inf処理: 1,305 → NaN
  Grouping: date
  Range: [-0.967256, 5.815371]
  1-99%ile: [-0.595606, 1.534475]
  5-95%ile: [-0.284501, 0.594025]

FF_SALES_CAGR_3Y:
  Total: 656,034
  Inf処理: 0 → NaN
  Grouping: date
  Range: [-0.660598, 1.970597]
  1-99%ile: [-0.364849, 0.785870]
  5-95%ile: [-0.176477, 0.377627]

FF_OPER_INC_YoY:
  Total: 656,034
  Inf処理: 465 → NaN
  Grouping: date
  Range: [-21.888540, 28.847458]
  1-99%ile: [-5.931932, 6.305607]
  5-95%ile: [-1.490814, 1.509728]

FF_OPER_INC_CAGR_3Y:
  Total: 656,034
  Inf処理: 0 → NaN
  Grouping: date
  Range: [-0.743737, 2.910003]
  1-99%ile: [-0.540780, 1.307888]
  5-95%ile: [-0.321919, 0.613309]

✅ クリーニング完了
📊 固定epsilon使用: 0.500000

🚀 加速度計算中...
  ✓ FF_SALES: epsilon=0.020000, 平均=0.0154, 標準偏差=1.7602
  ✓ FF_OPER_INC: epsilon=0.020000, 平均=-0.0469, 標準偏差=1.9824

✅ 完了！


,date,P_SYMBOL,variable,value
0,2006-08-31,0HSW-GB,FF_OPER_INC_CAGR_3Y,NaN
1,2006-08-31,0II3.XX1-GB,FF_OPER_INC_CAGR_3Y,NaN
2,2006-08-31,0MDJ-GB,FF_OPER_INC_CAGR_3Y,NaN
3,2006-08-31,0N1N-GB,FF_OPER_INC_CAGR_3Y,NaN
4,2006-08-31,0N3I-GB,FF_OPER_INC_CAGR_3Y,NaN
...,...,...,...,...
5247027,2025-10-31,ZBRA-US,FF_OPER_INC_Abs_Acc,0.010089
5247028,2025-10-31,ZM-US,FF_OPER_INC_Abs_Acc,NaN
5247029,2025-10-31,ZS-US,FF_OPER_INC_Abs_Acc,NaN
5247030,2025-10-31,ZTS-US,FF_OPER_INC_Abs_Acc,-0.072490


In [ ]:
df_slice = df_all_acc[df_all_acc["P_SYMBOL"] == "ZURN-CH"].reset_index(drop=True)
df_slice = df_slice.pivot(
    index=["date", "P_SYMBOL"], columns="variable", values="value"
).reset_index()

display(df_slice.select_dtypes(float).describe())


variable,FF_OPER_INC_Abs_Acc,FF_OPER_INC_CAGR_3Y,FF_OPER_INC_Rel_Acc,FF_OPER_INC_YoY,FF_SALES_Abs_Acc,FF_SALES_CAGR_3Y,FF_SALES_Rel_Acc,FF_SALES_YoY
count,181.000000,181.000000,181.000000,231.000000,201.000000,201.000000,201.000000,231.000000
mean,0.168240,0.109686,0.351724,0.212958,0.097263,0.066850,0.270573,0.140852
std,1.780770,0.364835,2.323262,1.795799,0.481826,0.161609,2.252922,0.498719
min,-3.980171,-0.564312,-3.000000,-3.915082,-0.593478,-0.316184,-3.000000,-0.638926
25%,-0.374363,-0.084801,-1.260525,-0.290736,-0.249372,-0.014981,-1.336416,-0.145586
50%,-0.024285,0.056592,-0.169046,0.051038,-0.013128,0.066319,-0.249055,0.016025
75%,0.446046,0.229812,3.000000,0.439211,0.286612,0.129072,3.000000,0.360254
max,10.275610,1.565719,3.000000,10.208607,1.622975,0.483872,3.000000,1.688396


In [ ]:
display(df_all.describe())


,date,value
count,2112222,2.112222e+06
mean,2016-10-02 20:32:17.946296320,NaN
min,2006-08-31 00:00:00,-inf
25%,2012-03-30 00:00:00,-4.700913e-02
50%,2016-10-31 00:00:00,2.079649e-02
75%,2021-04-30 00:00:00,1.510511e-01
max,2025-10-31 00:00:00,inf
std,NaN,NaN


#### 2. ファクターのランクを計算し、データベースに保存する。


### 4-3. マージン改善率


#### 1. QoQ, YoY, 3Yr, 5Yr の変化値（delta）を計算し、データベースに保存する。


In [ ]:
descriptor_list = [
    "FF_EBITDA_OPER_MGN",
    "FF_EBIT_OPER_MGN",
    "FF_NET_MGN",
    "FF_OPER_MGN",
    "FF_PTX_MGN",
    "FF_GROSS_MGN",
    "FF_ROA",
    "FF_ROE",
    "FF_ROIC",
    "FF_ROTC",
]


query = [
    f"SELECT `date`, `P_SYMBOL`, `variable`, `value` FROM `{table}`"
    for table in descriptor_list
]
query = " UNION ALL ".join(query)

period_list = ["CHANGE_QoQ", "CHANGE_YoY", "CHANGE_3Y", "CHANGE_5Y"]


In [ ]:
# ------------------------------------------------------
# load from database
# ------------------------------------------------------
with sqlite3.connect(financials_db_path) as conn:
    df_all = (
        pd.read_sql(query, con=conn, parse_dates=["date"])
        .sort_values("date", ignore_index=True)
        .astype({"variable": "category", "P_SYMBOL": "category"})
    ).sort_values(["variable", "P_SYMBOL", "date"], ignore_index=True)

display(df_all)
# groupby("variable")を使うことで、df_all全体を何度も走査(loc)するコストをゼロにする
# observed=True はcategory型を使う場合の高速化オプション
grouped = df_all.groupby("variable", observed=True)

# ------------------------------------------------------
# calculate growth and store to database
# ------------------------------------------------------
total_steps = len(descriptor_list)
for factor_name, df_descriptor in tqdm(grouped, total=total_steps, desc="Descriptors"):
    # df_factorはView(参照)の可能性があるため、計算用にコピーを作成
    # ここでメモリを食うが、factor単位なので全体コピーよりは軽い
    # かつ、ループの最後で解放される
    df_base = df_descriptor.copy()

    for growth in period_list:
        df_result = roic_utils.calculate_margin_delta(
            df=df_base, data_name=str(factor_name), growth_type=growth
        )

        # store to database
        new_variable_name = f"{factor_name}_{growth}_Ann"
        db_utils.delete_table_from_database(
            db_path=financials_db_path, table_name=new_variable_name
        )
        factset_utils.store_to_database(
            df=df_result,
            db_path=financials_db_path,
            table_name=new_variable_name,
            verbose=False,
        )

    # ---- メモリ管理 ----
    # 一つのファクター処理が終わったら、使用した一時変数を削除してGC実行
    del df_base
    gc.collect()


,date,P_SYMBOL,variable,value
0,2005-08-31,0HSW-GB,FF_EBITDA_OPER_MGN,9.473684
1,2005-09-30,0HSW-GB,FF_EBITDA_OPER_MGN,4.166667
2,2005-10-31,0HSW-GB,FF_EBITDA_OPER_MGN,4.166667
3,2005-11-30,0HSW-GB,FF_EBITDA_OPER_MGN,4.166667
4,2005-12-30,0HSW-GB,FF_EBITDA_OPER_MGN,NaN
...,...,...,...,...
7912625,2025-06-30,ZURN-CH,FF_ROTC,21.581542
7912626,2025-07-31,ZURN-CH,FF_ROTC,21.581542
7912627,2025-08-29,ZURN-CH,FF_ROTC,21.581542
7912628,2025-09-30,ZURN-CH,FF_ROTC,21.581542


Factors: 100%|██████████| 10/10 [19:25<00:00, 116.57s/it]


#### 2. Margin Improvement を計算し、データベースに保存する


In [ ]:
descriptor_list = [
    "FF_EBITDA_OPER_MGN",
    "FF_EBIT_OPER_MGN",
    "FF_NET_MGN",
    "FF_OPER_MGN",
    "FF_PTX_MGN",
    "FF_GROSS_MGN",
    "FF_ROA",
    "FF_ROE",
    "FF_ROIC",
    "FF_ROTC",
]


query = [
    f"SELECT `date`, `P_SYMBOL`, `variable`, `value` FROM `{table}`"
    for table in descriptor_list
]
query = " UNION ALL ".join(query)


In [ ]:
# ------------------------------------------------------
# load from database
# ------------------------------------------------------
with sqlite3.connect(financials_db_path) as conn:
    df_all = (
        pd.read_sql(
            query,
            con=conn,
            parse_dates=["date"],
        )
        .sort_values("date", ignore_index=True)
        .astype({"variable": "category", "P_SYMBOL": "category"})
    ).sort_values(["variable", "P_SYMBOL", "date"], ignore_index=True)

# groupby("variable")を使うことで、df_all全体を何度も走査(loc)するコストをゼロにする
# observed=True はcategory型を使う場合の高速化オプション
grouped = df_all.groupby("variable", observed=True)

growth_type_list = [
    "REL_PctCHANGE_QoQ",
    "REL_PctCHANGE_YoY",
    "REL_PctCHANGE_3Y",
    "REL_PctCHANGE_5Y",
]
total_steps = len(descriptor_list)
for descriptor_name, df_descriptor in tqdm(
    grouped, total=total_steps, desc="Descriptors"
):
    # df_factorはView(参照)の可能性があるため、計算用にコピーを作成
    # ここでメモリを食うが、factor単位なので全体コピーよりは軽い
    # かつ、ループの最後で解放される
    df_base = df_descriptor.copy()

    for growth_type in growth_type_list:
        df_result = roic_utils.calculate_margin_improvement_rate_relative(
            df=df_base, data_name=str(descriptor_name), growth_type=growth_type
        )

        # store to database
        new_variable_name = f"{descriptor_name}_{growth_type}"
        db_utils.delete_table_from_database(
            db_path=financials_db_path, table_name=new_variable_name
        )
        factset_utils.store_to_database(
            df=df_result,
            db_path=financials_db_path,
            table_name=new_variable_name,
            verbose=False,
        )

    # ---- メモリ管理 ----
    # 一つのファクター処理が終わったら、使用した一時変数を削除してGC実行
    del df_base
    gc.collect()


Descriptors: 100%|██████████| 10/10 [13:34<00:00, 81.44s/it]


#### 3. ファクターのランクを計算し、データベースに保存する


In [ ]:
factor_list = [
    "FF_EBITDA_OPER_MGN",
    "FF_EBIT_OPER_MGN",
    "FF_NET_MGN",
    "FF_OPER_MGN",
    "FF_PTX_MGN",
    "FF_GROSS_MGN",
    "FF_ROA",
    "FF_ROE",
    "FF_ROIC",
    "FF_ROTC",
]
period_list_1 = ["CHANGE_QoQ_Ann", "CHANGE_YoY_Ann", "CHANGE_3Y_Ann", "CHANGE_5Y_Ann"]
period_list_2 = [
    "REL_PctCHANGE_QoQ",
    "REL_PctCHANGE_YoY",
    "REL_PctCHANGE_3Y",
    "REL_PctCHANGE_5Y",
]


In [ ]:
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))
factset_utils.process_rank_calculation_store_to_db(
    df_weight=df_weight,
    factor_list=factor_list,
    financials_db_path=financials_db_path,
    period_list=period_list_1 + period_list_2,
)


🚀 処理開始: 40 件のタスク (Multi-Period Mode)


Rank計算進捗: 100%|██████████| 40/40 [36:26<00:00, 54.66s/it]  


🎉 全てのランク計算・保存が完了しました


### 4-4. Growth Factors


#### A. Scale Growth


##### A-1. データロード&欠損値確認 -> 欠損値はセクター中央値で補完


In [ ]:
# -----------------------------------
# ファクター計算のためのディスクリプターを指定
# -----------------------------------
descriptor_list = [
    "FF_SALES_YoY_PctRank_Sector_Neutral",
    "FF_SALES_CAGR_3Y_PctRank_Sector_Neutral",
    "FF_SALES_CAGR_5Y_PctRank_Sector_Neutral",
    "FF_OPER_INC_YoY_PctRank_Sector_Neutral",
    "FF_OPER_INC_CAGR_3Y_PctRank_Sector_Neutral",
    "FF_OPER_INC_CAGR_5Y_PctRank_Sector_Neutral",
]
df = load_factor_and_weight(descriptor_list=descriptor_list)
# -----------------------------------
# 欠損がない日付のみ使用
# -----------------------------------
valid_dates = df.groupby("date")[
    [
        "FF_SALES_CAGR_5Y_PctRank_Sector_Neutral",
        "FF_OPER_INC_CAGR_5Y_PctRank_Sector_Neutral",
    ]
].apply(lambda x: x.notna().any().any())  # type: ignore
valid_dates = valid_dates[valid_dates].index
df = df[df["date"].isin(valid_dates)].reset_index(drop=True)

# -----------------------------------
# 欠損値補完
# -----------------------------------
factset_utils.check_missing_value_and_fill_by_sector_median(
    df=df, factor_list=descriptor_list
)

# -----------------------------------
# データ確認
# -----------------------------------
display(df)
g = pd.DataFrame(df.groupby(["date"])["Weight (%)"].agg("sum"))
g_count = pd.DataFrame(df.groupby(["date"])["SEDOL"].count())
g_merged = pd.merge(g, g_count, left_index=True, right_index=True)

display(g_merged)


📋 欠損値の状況（補完前）
FF_SALES_YoY_PctRank_Sector_Neutral          :    341件 (  0.1%)
FF_SALES_CAGR_3Y_PctRank_Sector_Neutral      :  6,375件 (  2.7%)
FF_SALES_CAGR_5Y_PctRank_Sector_Neutral      : 12,255件 (  5.2%)
FF_OPER_INC_YoY_PctRank_Sector_Neutral       :     60件 (  0.0%)
FF_OPER_INC_CAGR_3Y_PctRank_Sector_Neutral   : 30,075件 ( 12.8%)
FF_OPER_INC_CAGR_5Y_PctRank_Sector_Neutral   : 36,134件 ( 15.3%)

⏳ セクター中央値で補完中...

📋 欠損値の状況（セクター中央値補完後）
✅ FF_SALES_YoY_PctRank_Sector_Neutral          :      0件 (  0.0%) | 補完: 341件
✅ FF_SALES_CAGR_3Y_PctRank_Sector_Neutral      :      0件 (  0.0%) | 補完: 6,375件
✅ FF_SALES_CAGR_5Y_PctRank_Sector_Neutral      :      0件 (  0.0%) | 補完: 12,255件
✅ FF_OPER_INC_YoY_PctRank_Sector_Neutral       :      0件 (  0.0%) | 補完: 60件
✅ FF_OPER_INC_CAGR_3Y_PctRank_Sector_Neutral   :      0件 (  0.0%) | 補完: 30,075件
✅ FF_OPER_INC_CAGR_5Y_PctRank_Sector_Neutral   :      0件 (  0.0%) | 補完: 36,134件

✅ 最終欠損値チェック
✅ FF_SALES_YoY_PctRank_Sector_Neutral          : 欠損なし
✅ FF_SALES_CAGR_3Y_PctR

,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),FF_OPER_INC_CAGR_3Y_PctRank_Sector_Neutral,FF_OPER_INC_CAGR_5Y_PctRank_Sector_Neutral,FF_OPER_INC_YoY_PctRank_Sector_Neutral,FF_SALES_CAGR_3Y_PctRank_Sector_Neutral,FF_SALES_CAGR_5Y_PctRank_Sector_Neutral,FF_SALES_YoY_PctRank_Sector_Neutral
0,2010-08-31,0N3I-GB,0896265,UKIDBM1,Tomkins PLC,Industrials,Capital Goods,0.024114,0.891429,0.673077,0.947368,0.908602,0.688889,0.780105
1,2010-08-31,0P7J-GB,0028262,UKIAPY1,Amec Foster Wheeler plc,Energy,Energy,0.025667,0.905263,0.775000,0.461538,0.300000,0.064516,0.339806
2,2010-08-31,1-HK,6190273,HKGAAE1,CKハチソン・ホールディングス,Financials,Real Estate,0.095978,0.666667,0.872038,0.689531,0.966667,0.976654,0.917563
3,2010-08-31,10-HK,6408352,HKGAGG1,Hang Lung Group Limited,Financials,Real Estate,0.026328,0.502193,0.407583,0.602888,0.548148,0.210117,0.688172
4,2010-08-31,101-HK,6030506,HKGAAA1,ハンルン・プロパティーズ,Financials,Real Estate,0.050837,0.502193,0.502370,0.638989,0.492593,0.435798,0.684588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235818,2025-10-31,ZBRA-US,2989356,USAP8H1,ゼブラ・テクノロジーズ・コーポレーション Class A,Information Technology,Technology Hardware & Equipment,0.017637,0.366197,0.205479,0.353982,0.190476,0.154762,0.283186
235819,2025-10-31,ZM-US,BGSP7M9,USBEOV1,ズーム・ビデオ・コミュニケーションズ Class A,Information Technology,Software & Services,0.027782,0.507042,0.506849,0.902655,0.505952,0.505952,0.159292
235820,2025-10-31,ZS-US,BZ00V34,USBDYI1,ゼットスケイラー,Information Technology,Software & Services,0.043166,0.507042,0.506849,0.168142,0.505952,0.505952,0.637168
235821,2025-10-31,ZTS-US,B95WG16,USBANZ1,ゾエティス Class A,Health Care,Pharmaceuticals Biotechnology & Life Sciences,0.082630,0.597826,0.488636,0.314815,0.459184,0.322917,0.074074


,Weight (%),SEDOL
date,,
2010-08-31,99.783547,1307
2010-09-30,99.814447,1307
2010-10-31,99.805734,1308
2010-11-30,99.769463,1307
2010-12-31,99.634624,1306
...,...,...
2025-06-30,99.999989,1142
2025-07-31,100.000012,1139
2025-08-31,100.000009,1140


##### A-2. Scale Growth Factor 計算


In [ ]:
# -----------------------------------
# Composite Growth Factorの計算
# -----------------------------------
# ウェイト設定
blend_weight = {
    "FF_SALES_YoY_PctRank_Sector_Neutral": 1 / 6,
    "FF_SALES_CAGR_3Y_PctRank_Sector_Neutral": 1 / 6,
    "FF_SALES_CAGR_5Y_PctRank_Sector_Neutral": 1 / 6,
    "FF_OPER_INC_YoY_PctRank_Sector_Neutral": 1 / 6,
    "FF_OPER_INC_CAGR_3Y_PctRank_Sector_Neutral": 1 / 6,
    "FF_OPER_INC_CAGR_5Y_PctRank_Sector_Neutral": 1 / 6,
}
factor_name = "Factor_Scale_Growth"
df = factset_utils.create_factor(
    df=df, factor_name=factor_name, blend_weight=blend_weight
)
# -----------------------------------
# Store to database
# -----------------------------------
for variable in [f"{factor_name}_Score", f"{factor_name}_Score_Rank"]:
    df_slice = (
        df[["date", "P_SYMBOL", variable]]
        .assign(variable=variable, date=lambda x: pd.to_datetime(x["date"]))
        .rename(columns={variable: "value"})
    )
    db_utils.delete_table_from_database(db_path=financials_db_path, table_name=variable)
    factset_utils.store_to_database(
        df=df_slice,
        db_path=financials_db_path,
        table_name=variable,
        verbose=True,
    )



📊 Factor 計算
ウェイト設定:
  FF_SALES_YoY_PctRank_Sector_Neutral          : 16.7%
  FF_SALES_CAGR_3Y_PctRank_Sector_Neutral      : 16.7%
  FF_SALES_CAGR_5Y_PctRank_Sector_Neutral      : 16.7%
  FF_OPER_INC_YoY_PctRank_Sector_Neutral       : 16.7%
  FF_OPER_INC_CAGR_3Y_PctRank_Sector_Neutral   : 16.7%
  FF_OPER_INC_CAGR_5Y_PctRank_Sector_Neutral   : 16.7%

✅ Score 計算完了
   平均: 0.5044
   標準偏差: 0.2007
   最小値: 0.0086
   最大値: 1.0000

📊 Rank 分布:
  rank5: 47,240件 ( 20.0%)
  rank4: 47,126件 ( 20.0%)
  rank3: 47,131件 ( 20.0%)
  rank2: 47,126件 ( 20.0%)
  rank1: 47,200件 ( 20.0%)

🎯 最終データサンプル:
             date P_SYMBOL    SEDOL             GICS Sector  \
235803 2025-10-31  WTRG-US  BLCF3J9               Utilities   
235804 2025-10-31   WTW-US  BDB6Q21              Financials   
235805 2025-10-31    WY-US  2958936             Real Estate   
235806 2025-10-31     X-CA  B8KH5G7              Financials   
235807 2025-10-31   XEL-US  2614807               Utilities   
235808 2025-10-31   XOM-US  2326618      

#### B. Margin Improvement


- 1, 3, 5 年のマージン変化幅
- 1, 3, 5 年のマージン改善率


##### B-1. データロード&欠損値確認 -> 欠損値はセクター中央値で補完


In [ ]:
# ファクター計算のためのディスクリプターを指定
descriptor_list = [
    "FF_OPER_MGN_CHANGE_YoY_Ann_PctRank_Sector_Neutral",
    # "FF_OPER_MGN_CHANGE_3Y_Ann_PctRank_Sector_Neutral",
    # "FF_OPER_MGN_CHANGE_5Y_Ann_PctRank_Sector_Neutral",
    "FF_OPER_MGN_REL_PctCHANGE_YoY_PctRank_Sector_Neutral",
    # "FF_OPER_MGN_REL_PctCHANGE_3Y_PctRank_Sector_Neutral",
    # "FF_OPER_MGN_REL_PctCHANGE_5Y_PctRank_Sector_Neutral",
    "FF_NET_MGN_CHANGE_YoY_Ann_PctRank_Sector_Neutral",
    # "FF_NET_MGN_CHANGE_3Y_Ann_PctRank_Sector_Neutral",
    # "FF_NET_MGN_CHANGE_5Y_Ann_PctRank_Sector_Neutral",
    "FF_NET_MGN_REL_PctCHANGE_YoY_PctRank_Sector_Neutral",
    # "FF_NET_MGN_REL_PctCHANGE_3Y_PctRank_Sector_Neutral",
    # "FF_NET_MGN_REL_PctCHANGE_5Y_PctRank_Sector_Neutral",
]


In [ ]:
df = load_factor_and_weight(descriptor_list=descriptor_list)
# -----------------------------------
# 欠損がない日付のみ使用
# -----------------------------------
valid_dates = df.groupby("date")[
    [
        "FF_OPER_MGN_CHANGE_5Y_Ann_PctRank_Sector_Neutral",
        "FF_NET_MGN_CHANGE_5Y_Ann_PctRank_Sector_Neutral",
    ]
].apply(lambda x: x.notna().any().any())  # type: ignore
valid_dates = valid_dates[valid_dates].index
df = df[df["date"].isin(valid_dates)].reset_index(drop=True)

# -----------------------------------
# 欠損値補完
# -----------------------------------
factset_utils.check_missing_value_and_fill_by_sector_median(
    df=df, factor_list=descriptor_list
)

# -----------------------------------
# データ確認
# -----------------------------------
display(df)
g = pd.DataFrame(df.groupby(["date"])["Weight (%)"].agg("sum"))
g_count = pd.DataFrame(df.groupby(["date"])["SEDOL"].count())
g_merged = pd.merge(g, g_count, left_index=True, right_index=True)

display(g_merged)


#### B-2. Margin Grwoth Factor 計算


In [ ]:
# -----------------------------------
# Factorの計算
# -----------------------------------
# ウェイト設定
blend_weight_1 = {
    "FF_OPER_MGN_CHANGE_YoY_Ann_PctRank_Sector_Neutral": 1 / 12,
    "FF_OPER_MGN_CHANGE_3Y_Ann_PctRank_Sector_Neutral": 1 / 12,
    "FF_OPER_MGN_CHANGE_5Y_Ann_PctRank_Sector_Neutral": 1 / 12,
    "FF_OPER_MGN_REL_PctCHANGE_YoY_PctRank_Sector_Neutral": 1 / 12,
    "FF_OPER_MGN_REL_PctCHANGE_3Y_PctRank_Sector_Neutral": 1 / 12,
    "FF_OPER_MGN_REL_PctCHANGE_5Y_PctRank_Sector_Neutral": 1 / 12,
    "FF_NET_MGN_CHANGE_YoY_Ann_PctRank_Sector_Neutral": 1 / 12,
    "FF_NET_MGN_CHANGE_3Y_Ann_PctRank_Sector_Neutral": 1 / 12,
    "FF_NET_MGN_CHANGE_5Y_Ann_PctRank_Sector_Neutral": 1 / 12,
    "FF_NET_MGN_REL_PctCHANGE_YoY_PctRank_Sector_Neutral": 1 / 12,
    "FF_NET_MGN_REL_PctCHANGE_3Y_PctRank_Sector_Neutral": 1 / 12,
    "FF_NET_MGN_REL_PctCHANGE_5Y_PctRank_Sector_Neutral": 1 / 12,
}
blend_weight_2 = {
    "FF_OPER_MGN_CHANGE_YoY_Ann_PctRank_Sector_Neutral": 0.25,
    "FF_OPER_MGN_REL_PctCHANGE_YoY_PctRank_Sector_Neutral": 0.25,
    "FF_NET_MGN_CHANGE_YoY_Ann_PctRank_Sector_Neutral": 0.25,
    "FF_NET_MGN_REL_PctCHANGE_YoY_PctRank_Sector_Neutral": 0.25,
}
factor_name = "Factor_Margin_Growth_YoY"
df = factset_utils.create_factor(
    df=df, factor_name=factor_name, blend_weight=blend_weight_2
)
# -----------------------------------
# Store to database
# -----------------------------------
for variable in [f"{factor_name}_Score", f"{factor_name}_Score_Rank"]:
    df_slice = (
        df[["date", "P_SYMBOL", variable]]
        .assign(variable=variable, date=lambda x: pd.to_datetime(x["date"]))
        .rename(columns={variable: "value"})
    )
    db_utils.delete_table_from_database(db_path=financials_db_path, table_name=variable)
    factset_utils.store_to_database(
        df=df_slice,
        db_path=financials_db_path,
        table_name=variable,
        verbose=True,
    )



📊 Factor 計算
ウェイト設定:
  FF_OPER_MGN_CHANGE_YoY_Ann_PctRank_Sector_Neutral: 25.0%
  FF_OPER_MGN_REL_PctCHANGE_YoY_PctRank_Sector_Neutral: 25.0%
  FF_NET_MGN_CHANGE_YoY_Ann_PctRank_Sector_Neutral: 25.0%
  FF_NET_MGN_REL_PctCHANGE_YoY_PctRank_Sector_Neutral: 25.0%


KeyError: 'FF_OPER_MGN_CHANGE_YoY_Ann_PctRank_Sector_Neutral'

### 4-5. Valuation（Bloomberg） -> ⚠️ データ不足のためスキップ


`BEST_EPS`と`TRAIL_12M_EPS_BEF_XO_ITEM`の QoQ, YoY, 3Yr CAGR, 5Yr CAGR の値を計算し、データベースに保存する。


In [ ]:
data_list = ["BEST_PE_RATIO", "BEST_EPS", "PE_RATIO", "TRAIL_12M_EPS_BEF_XO_ITEM"]

blp = bloomberg_utils.BlpapiCustom()

with sqlite3.connect(bloomberg_valuation_db_path) as conn:
    for data in tqdm(data_list):
        df = (
            pd.read_sql(f"SELECT * FROM `{data}`", con=conn, parse_dates=["Date"])
            .sort_values("Date", ignore_index=True)
            .rename(columns={"Date": "date", "SEDOL": "P_SYMBOL"})
        )  # calculate_growth関数を使うために一時的にリネーム

        for growth in ["QoQ", "YoY", "CAGR_3Y", "CAGR_5Y"]:
            df_growth = df.copy()
            new_variable = f"{data}_{growth}"

            df_growth = roic_utils.calculate_growth(
                df=df_growth, data_name=data, growth_type=growth
            ).rename(
                columns={"date": "Date", "P_SYMBOL": "SEDOL"}
            )  # 元のカラム名に戻す

            db_utils.delete_table_from_database(
                db_path=bloomberg_valuation_db_path, table_name=new_variable
            )

            blp.store_to_database(
                df=df_growth,
                db_path=bloomberg_valuation_db_path,
                table_name=new_variable,
                primary_keys=["date", "SEDOL", "variable"],
            )


  0%|          | 0/4 [00:00<?, ?it/s]

✅ 保存完了。テーブル 'BEST_PE_RATIO_QoQ' に 55043 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'BEST_PE_RATIO_YoY' に 38534 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'BEST_PE_RATIO_CAGR_3Y' に 10608 行を処理しました (IGNORE)。


 25%|██▌       | 1/4 [00:13<00:39, 13.07s/it]

✅ 保存完了。テーブル 'BEST_PE_RATIO_CAGR_5Y' に 889 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'BEST_EPS_QoQ' に 61342 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'BEST_EPS_YoY' に 41722 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'BEST_EPS_CAGR_3Y' に 7895 行を処理しました (IGNORE)。


 50%|█████     | 2/4 [00:25<00:25, 12.51s/it]

✅ 保存完了。テーブル 'BEST_EPS_CAGR_5Y' に 122 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'PE_RATIO_QoQ' に 512335 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'PE_RATIO_YoY' に 485879 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'PE_RATIO_CAGR_3Y' に 419875 行を処理しました (IGNORE)。


 75%|███████▌  | 3/4 [02:43<01:09, 69.83s/it]

✅ 保存完了。テーブル 'PE_RATIO_CAGR_5Y' に 359299 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'TRAIL_12M_EPS_BEF_XO_ITEM_QoQ' に 161910 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'TRAIL_12M_EPS_BEF_XO_ITEM_YoY' に 136498 行を処理しました (IGNORE)。
✅ 保存完了。テーブル 'TRAIL_12M_EPS_BEF_XO_ITEM_CAGR_3Y' に 61627 行を処理しました (IGNORE)。


100%|██████████| 4/4 [03:25<00:00, 51.43s/it]

✅ 保存完了。テーブル 'TRAIL_12M_EPS_BEF_XO_ITEM_CAGR_5Y' に 32107 行を処理しました (IGNORE)。


In [ ]:
with sqlite3.connect(bloomberg_valuation_db_path) as conn:
    df = pd.read_sql("SELECT * FROM BEST_EPS", con=conn, parse_dates=["Date"])
    display(df)


,Date,SEDOL,variable,value
0,2000-01-31,2206301,BEST_EPS,0.820
1,2000-01-31,2213981,BEST_EPS,3.653
2,2000-01-31,2226536,BEST_EPS,0.033
3,2000-01-31,2350651,BEST_EPS,0.230
4,2000-01-31,2552275,BEST_EPS,0.220
...,...,...,...,...
68844,2025-10-31,BZ8GX83,BEST_EPS,0.343
68845,2025-10-31,BZ8VJQ8,BEST_EPS,2.270
68846,2025-10-31,BZBW6G7,BEST_EPS,0.196
68847,2025-10-31,BZBYG74,BEST_EPS,0.216


In [ ]:
tables = db_utils.get_table_names(bloomberg_valuation_db_path)
display(tables)


['BEST_PE_RATIO',
 'BEST_EPS',
 'PE_RATIO',
 'TRAIL_12M_EPS_BEF_XO_ITEM',
 'BEST_PE_RATIO_QoQ',
 'BEST_PE_RATIO_YoY',
 'BEST_PE_RATIO_CAGR_3Y',
 'BEST_PE_RATIO_CAGR_5Y',
 'BEST_EPS_QoQ',
 'BEST_EPS_YoY',
 'BEST_EPS_CAGR_3Y',
 'BEST_EPS_CAGR_5Y',
 'PE_RATIO_QoQ',
 'PE_RATIO_YoY',
 'PE_RATIO_CAGR_3Y',
 'PE_RATIO_CAGR_5Y',
 'TRAIL_12M_EPS_BEF_XO_ITEM_QoQ',
 'TRAIL_12M_EPS_BEF_XO_ITEM_YoY',
 'TRAIL_12M_EPS_BEF_XO_ITEM_CAGR_3Y',
 'TRAIL_12M_EPS_BEF_XO_ITEM_CAGR_5Y']

In [ ]:
# 構成銘柄情報
query = f"""
    SELECT
        `date`, `P_SYMBOL`, `SEDOL`, `FG_COMPANY_NAME`, `Asset ID`, `GICS Sector`, `Weight (%)`
    FROM
        {UNIVERSE_CODE}
"""
with sqlite3.connect(factset_index_db_path) as conn:
    df_weight = pd.read_sql(query, parse_dates=["date"], con=conn)

# ファクター値
factor_list = ["BEST_EPS", "BEST_PE_RATIO"]

with sqlite3.connect(bloomberg_valuation_db_path) as conn:
    # total引数を追加
    for factor in tqdm(factor_list):
        # データベースから呼び出し
        df = (
            pd.read_sql(
                f"SELECT `Date`, `SEDOL`, `value` FROM `{factor}`",
                con=conn,
                parse_dates=["Date"],
            )
            .rename(columns={"Date": "date"})
            .assign(
                date=lambda row: pd.to_datetime(row["date"])
                + pd.tseries.offsets.MonthEnd(0)
            )
            .sort_values("date", ignore_index=True)
            .rename(columns={"value": factor})
        )

        # merge: 構成銘柄情報とファクター
        df = (
            pd.merge(df_weight, df, on=["date", "SEDOL"], how="outer")
            .drop_duplicates(subset=["date", "SEDOL"])
            .dropna(
                subset=["Weight (%)", factor],
                how="any",
                axis=0,
                ignore_index=True,
            )
        )

        g = df.groupby(["date"])["Weight (%)"].agg("sum").to_frame()
        display(g.tail(50))


In [ ]:
# 構成銘柄情報
query = f"""
    SELECT
        `date`, `P_SYMBOL`, `SEDOL`, `FG_COMPANY_NAME`, `Asset ID`, `GICS Sector`, `Weight (%)`
    FROM
        {UNIVERSE_CODE}
"""
with sqlite3.connect(factset_index_db_path) as conn:
    df_weight = pd.read_sql(query, parse_dates=["date"], con=conn)

# ファクター値
factor_list = ["BEST_EPS", "BEST_PE_RATIO"]
period_list = ["QoQ", "YoY", "CAGR_3Y", "CAGR_5Y"]
total_iterations = len(factor_list) * len(period_list)
print(f"処理総数: {total_iterations} 回")

with sqlite3.connect(bloomberg_valuation_db_path) as conn:
    # total引数を追加
    for factor, periods in tqdm(
        itertools.product(factor_list, period_list),
        total=total_iterations,
        desc="ファクター処理進捗",
    ):
        factor_growth = f"{factor}_{periods}"
        # データベースから呼び出し
        df = (
            pd.read_sql(
                f"SELECT `Date`, `SEDOL`, `value` FROM `{factor_growth}`",
                con=conn,
                parse_dates=["Date"],
            )
            .rename(columns={"Date": "date"})
            .assign(
                date=lambda row: pd.to_datetime(row["date"])
                + pd.tseries.offsets.MonthEnd(0)
            )
            .sort_values("date", ignore_index=True)
            .rename(columns={"value": factor_growth})
        )

        # merge: 構成銘柄情報とファクター
        df = (
            pd.merge(df_weight, df, on=["date", "SEDOL"], how="outer")
            .drop_duplicates(subset=["date", "SEDOL"])
            .dropna(
                subset=["Weight (%)", factor_growth],
                how="any",
                axis=0,
                ignore_index=True,
            )
        )
        display(df)


### 4-6. ROIC Label Factor（Factset）


ROIC(ROE) + Security Code

- **セクター中立あり・なし 両方を計算**
- 金融セクターのみ ROIC の代わりに ROE を使用（ただしデータフレームのカラム名は ROIC で表記）


In [ ]:
# ----------------------------------------
# 1. get ROIC and ROE data
# 2. get security info(Index constituents)
# ----------------------------------------
df_roic_and_roe = factset_utils.load_financial_data(
    financials_db_path=financials_db_path, factor_list=["FF_ROIC", "FF_ROE"]
)
security_info = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)

# ----------------------------------------
# merge
# ----------------------------------------
df_roic_merged = (
    factset_utils.merge_idx_constituents_and_financials(
        df_weight=security_info, df_factor=df_roic_and_roe
    )
    .assign(
        ROIC=lambda x: np.where(
            x["GICS Sector"] == "Financials", x["FF_ROE"], x["FF_ROIC"]
        )
    )
    .dropna(subset=["Weight (%)", "ROIC"], how="any", ignore_index=True)
    .drop(columns=["FF_ROIC", "FF_ROE"])
)


In [ ]:
year_period_list = [3, 5]  # year_periodまでさかのぼってROICの移行を見る
sector_neutral_mode_list = [True, False]  # sector neutralにするかしないか

for year_period, sector_neutral_mode in itertools.product(
    year_period_list, sector_neutral_mode_list
):
    df = df_roic_merged.copy()
    # ----------------------------------------
    # ROICをランキング
    # ----------------------------------------
    df = roic_utils.add_factor_rank_cols(
        df, factor_name="ROIC", sector_neutral_mode=sector_neutral_mode
    ).rename(columns={"P_SYMBOL": "Symbol"})

    roic_rank_name = "ROIC_Rank_Sector_Neutral" if sector_neutral_mode else "ROIC_Rank"
    df = roic_utils.add_shifted_factor_cols_month(
        df,
        factor_name=roic_rank_name,
        shift_month=list(range(1, int(year_period * 12) + 1)),
        shift_direction="Past",
    ).rename(columns={"Symbol": "P_SYMBOL"})

    # ----------------------------------------
    # ROICラベルを付与
    # ----------------------------------------
    roic_transition_name = (
        f"ROIC_label_Past{year_period}Y_Sector_Neutral"
        if sector_neutral_mode
        else f"ROIC_label_Past{year_period}Y"
    )
    df[roic_transition_name] = df.apply(
        lambda row: roic_utils.make_roic_transition_label(
            row=row,
            roic_rank_col=roic_rank_name,
            freq="annual",
            shift_direction="Past",
            year_period=year_period,
            judge_by_slope=False,
        ),
        axis=1,
    )

    df = (
        df[["date", "P_SYMBOL", roic_transition_name]]
        .astype({"P_SYMBOL": "category", roic_transition_name: "category"})
        .rename(columns={roic_transition_name: "value"})
        .dropna(subset=["value"], ignore_index=True)
        .assign(
            variable=roic_transition_name, date=lambda row: pd.to_datetime(row["date"])
        )
        .reindex(columns=["date", "P_SYMBOL", "variable", "value"])
    )

    # ----------------------------------------
    # データベース保存
    # ----------------------------------------
    db_utils.delete_table_from_database(
        db_path=financials_db_path, table_name=roic_transition_name, verbose=True
    )
    factset_utils.store_to_database(
        df=df, db_path=financials_db_path, table_name=roic_transition_name
    )
    del df
    gc.collect()


データベース 'Financials_and_Price.db' からテーブル 'ROIC_label_Past3Y_Sector_Neutral' を削除しました（または存在しませんでした）。
既存の 0 行との重複をチェックしました。220526 行を新たに追加します。
  -> ROIC_label_Past3Y_Sector_Neutral: データの書き込みが完了しました。
データベース 'Financials_and_Price.db' からテーブル 'ROIC_label_Past3Y' を削除しました（または存在しませんでした）。
既存の 0 行との重複をチェックしました。220526 行を新たに追加します。
  -> ROIC_label_Past3Y: データの書き込みが完了しました。
データベース 'Financials_and_Price.db' からテーブル 'ROIC_label_Past5Y_Sector_Neutral' を削除しました（または存在しませんでした）。
既存の 0 行との重複をチェックしました。176449 行を新たに追加します。
  -> ROIC_label_Past5Y_Sector_Neutral: データの書き込みが完了しました。
データベース 'Financials_and_Price.db' からテーブル 'ROIC_label_Past5Y' を削除しました（または存在しませんでした）。
既存の 0 行との重複をチェックしました。176449 行を新たに追加します。
  -> ROIC_label_Past5Y: データの書き込みが完了しました。


In [ ]:
df_roic_and_roe = factset_utils.load_financial_data(
    financials_db_path=financials_db_path,
    factor_list=[
        "ROIC_label_Past3Y",
        "ROIC_label_Past3Y_Sector_Neutral",
        "ROIC_label_Past5Y",
        "ROIC_label_Past5Y_Sector_Neutral",
    ],
)
security_info = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)

# ----------------------------------------
# merge
# ----------------------------------------
df = factset_utils.merge_idx_constituents_and_financials(
    df_weight=security_info, df_factor=df_roic_and_roe
).dropna(subset=["Weight (%)"], ignore_index=True)

roic_label_name = "ROIC_label_Past5Y_Sector_Neutral"


# --- label count ---
roic_count = pd.pivot(
    pd.DataFrame(
        df.groupby(["date", "GICS Sector", roic_label_name])["P_SYMBOL"].count()
    ).reset_index(),
    index=["date", "GICS Sector"],
    columns=roic_label_name,
).reset_index()
display(roic_count.loc[roic_count["GICS Sector"] == "Information Technology"])

# --- weight check ---
weight_total_count = (
    df.groupby(["date"])["Weight (%)"]
    .agg(["count", "sum"])
    .rename(columns={"count": "Num of Securities", "sum": "Total Weight (%)"})
).sort_index()

weight_sector_count = (
    df.groupby(["date", "GICS Sector"])["Weight (%)"]
    .agg(["count", "sum"])
    .rename(columns={"count": "Num of Securities", "sum": "Total Weight (%)"})
).sort_index()
display(weight_total_count)
display(weight_sector_count)

roic_label_count = (
    df.groupby(["date", roic_label_name])["Weight (%)"]
    .agg(["count", "sum"])
    .rename(columns={"count": "Num of Securities", "sum": "Total Weight (%)"})
).sort_index()
display(roic_label_count)


date             GICS Sector  \
ROIC_label_Past5Y_Sector_Neutral                                      
23                               2010-08-31  Information Technology   
33                               2010-09-30  Information Technology   
43                               2010-10-31  Information Technology   
53                               2010-11-30  Information Technology   
63                               2010-12-31  Information Technology   
...                                     ...                     ...   
1908                             2025-06-30  Information Technology   
1919                             2025-07-31  Information Technology   
1930                             2025-08-31  Information Technology   
1941                             2025-09-30  Information Technology   
1952                             2025-10-31  Information Technology   

                                    P_SYMBOL                                  \
ROIC_label_Past5Y_Sector_Neutral drop to low move to high others remain high   
23                                       8.0         14.0   16.0        11.0   
33                                       6.0         13.0   20.0        11.0   
43                                       6.0         12.0   21.0        11.0   
53                                       6.0         13.0   21.0        12.0   
63                                       5.0         13.0   20.0        13.0   
...                                      ...          ...    ...         ...   
1908                                    12.0         10.0   36.0        24.0   
1919                                    15.0         13.0   32.0        23.0   
1930                                    14.0         11.0   33.0        24.0   
1941                                    13.0         12.0   28.0        25.0   
1952                                    11.0         11.0   20.0        19.0   

                                             
ROIC_label_Past5Y_Sector_Neutral remain low  
23                                      4.0  
33                                      5.0  
43                                      5.0  
53                                      5.0  
63                                      6.0  
...                                     ...  
1908                                    9.0  
1919                                    7.0  
1930                                    7.0  
1941                                    8.0  
1952                                    4.0  

[183 rows x 7 columns]

,Num of Securities,Total Weight (%)
date,,
2000-01-31,1065,100.000000
2000-02-29,1062,99.999994
2000-03-31,1057,100.000025
2000-04-30,1055,100.000019
2000-05-31,1046,100.000018
...,...,...
2025-06-30,1142,99.999989
2025-07-31,1139,100.000012
2025-08-31,1140,100.000009


Num of Securities  Total Weight (%)
date       GICS Sector                                                
2000-01-31 Communication Services                 32         11.771133
           Consumer Discretionary                187         13.164694
           Consumer Staples                       88          6.224660
           Energy                                 38          5.651133
           Financials                            172         16.592643
...                                              ...               ...
2025-10-31 Industrials                           209         10.274341
           Information Technology                113         29.397901
           Materials                              81          3.080950
           Real Estate                            63          1.810682
           Utilities                              68          2.693171

[3210 rows x 2 columns]

Num of Securities  \
date       ROIC_label_Past5Y_Sector_Neutral                      
2009-04-30 remain high                                       2   
2009-05-31 remain high                                       2   
2009-06-30 others                                            2   
2009-07-31 others                                            2   
2009-08-31 others                                            2   
...                                                        ...   
2025-10-31 drop to low                                     127   
           move to high                                    152   
           others                                          375   
           remain high                                     145   
           remain low                                       96   

                                             Total Weight (%)  
date       ROIC_label_Past5Y_Sector_Neutral                    
2009-04-30 remain high                               0.612970  
2009-05-31 remain high                               0.621910  
2009-06-30 others                                    0.637049  
2009-07-31 others                                    0.621194  
2009-08-31 others                                    0.642319  
...                                                       ...  
2025-10-31 drop to low                               8.286072  
           move to high                              9.589011  
           others                                   27.350548  
           remain high                              30.448917  
           remain low                                4.115467  

[931 rows x 2 columns]

### （🚧 工事中）4-7. ROIC 分位移動


- 3 年前の ROIC5 分位 -> 現在の ROIC5 分位への移動をラベリング
- Financials セクターは ROIC の代わりに ROE 使用


In [ ]:
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)

df_factor = factset_utils.load_financial_data(
    financials_db_path=financials_db_path, factor_list=["FF_ROIC_Rank", "FF_ROE_Rank"]
)

df_roic_rank = (
    factset_utils.merge_idx_constituensts_and_financials(
        df_weight=df_weight, df_factor=df_factor
    )
    .assign(
        ROIC_Rank=lambda x: np.where(
            x["GICS Sector"] == "Financials", x["FF_ROE_Rank"], x["FF_ROIC_Rank"]
        )
    )
    .drop(columns=["FF_ROIC_Rank", "FF_ROE_Rank"])
    .rename(columns={"SEDOL": "Symbol"})
)
df_roic_rank = (
    roic_utils.add_shifted_factor_cols_month(
        df_roic_rank,
        factor_name="ROIC_Rank",
        shift_month=[36],
        shift_direction="Past",
    )
    .rename(columns={"Symbol": "SEDOL"})
    .dropna(subset=["ROIC_Rank", "ROIC_Rank_36MAgo"], how="any")
    .sort_values(["SEDOL", "date"], ignore_index=True)
)

display(df_roic_rank)


,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),ROIC_Rank,ROIC_Rank_36MAgo
0,2015-06-30,ADN-GB,0003128,UKIDUK1,Aberdeen Asset Management PLC,Financials,Financial Services,0.019301,rank1,rank1
1,2015-07-31,ADN-GB,0003128,UKIDUK1,Aberdeen Asset Management PLC,Financials,Financial Services,0.016965,rank1,rank1
2,2015-08-31,ADN-GB,0003128,UKIDUK1,Aberdeen Asset Management PLC,Financials,Financial Services,0.015731,rank1,rank1
3,2015-09-30,ADN-GB,0003128,UKIDUK1,Aberdeen Asset Management PLC,Financials,Financial Services,0.014948,rank1,rank1
4,2015-10-31,ADN-GB,0003128,UKIDUK1,Aberdeen Asset Management PLC,Financials,Financial Services,0.016557,rank1,rank1
...,...,...,...,...,...,...,...,...,...,...
206145,2025-06-30,DHER-DE,BZCNB42,GER2BR1,デリバリー・ヒーロー,Consumer Discretionary,Consumer Services,0.007228,rank5,rank5
206146,2025-07-31,DHER-DE,BZCNB42,GER2BR1,デリバリー・ヒーロー,Consumer Discretionary,Consumer Services,0.007958,rank5,rank5
206147,2025-08-31,DHER-DE,BZCNB42,GER2BR1,デリバリー・ヒーロー,Consumer Discretionary,Consumer Services,0.006908,rank5,rank5
206148,2025-09-30,DHER-DE,BZCNB42,GER2BR1,デリバリー・ヒーロー,Consumer Discretionary,Consumer Services,0.007233,rank5,rank5


In [ ]:
g = pd.DataFrame(df_roic_rank.groupby("date")["Weight (%)"].agg("sum"))
display(g)

g_count = (
    pd.DataFrame(df_roic_rank.groupby(["date", "GICS Sector"])["SEDOL"].count())
    .reset_index()
    .pivot(index=["date"], columns="GICS Sector", values="SEDOL")
)
display(g_count)


,Weight (%)
date,
2008-08-31,74.801052
2008-09-30,76.101417
2008-10-31,76.204146
2008-11-30,76.204061
2008-12-31,80.869285
...,...
2025-06-30,93.851450
2025-07-31,94.125253
2025-08-31,93.974811


GICS Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
date,,,,,,,,,,,
2008-08-31,30.0,131.0,63.0,57.0,193.0,59.0,124.0,69.0,63.0,NaN,44.0
2008-09-30,31.0,126.0,61.0,58.0,190.0,63.0,123.0,70.0,65.0,NaN,41.0
2008-10-31,31.0,127.0,61.0,59.0,190.0,63.0,122.0,70.0,65.0,NaN,42.0
2008-11-30,31.0,117.0,60.0,59.0,183.0,62.0,119.0,68.0,65.0,NaN,42.0
2008-12-31,28.0,131.0,64.0,58.0,183.0,67.0,121.0,72.0,65.0,NaN,48.0
...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,50.0,92.0,79.0,43.0,179.0,96.0,172.0,99.0,65.0,60.0,64.0
2025-07-31,50.0,92.0,79.0,43.0,179.0,97.0,172.0,98.0,65.0,60.0,64.0
2025-08-31,49.0,91.0,77.0,43.0,179.0,95.0,172.0,97.0,64.0,60.0,64.0


In [ ]:
g_roic_rank_shift = (
    pd.DataFrame(
        df_roic_rank.groupby(["date", "ROIC_Rank", "ROIC_Rank_36MAgo"])["SEDOL"].count()
    )
    .reset_index()
    .rename(columns={"SEDOL": "n_SEDOL"})
)
g_sedol_count_present_rank = (
    pd.DataFrame(g_roic_rank_shift.groupby(["date", "ROIC_Rank"])["n_SEDOL"].sum())
    .reset_index()
    .rename(columns={"n_SEDOL": "n_SEDOL_ROIC_Rank"})
)

g_sedol_count_past_rank = (
    pd.DataFrame(
        g_roic_rank_shift.groupby(["date", "ROIC_Rank_36MAgo"])["n_SEDOL"].sum()
    )
    .reset_index()
    .rename(columns={"n_SEDOL": "n_SEDOL_ROIC_Rank_36MAgo"})
)

g_roic_rank_shift = pd.merge(
    g_roic_rank_shift, g_sedol_count_present_rank, on=["date", "ROIC_Rank"], how="left"
)
g_roic_rank_shift = pd.merge(
    g_roic_rank_shift,
    g_sedol_count_past_rank,
    on=["date", "ROIC_Rank_36MAgo"],
    how="left",
)

g_roic_rank_shift = g_roic_rank_shift.assign(
    n_SEDOL_ROIC_Rank_pct=lambda x: x["n_SEDOL"].div(x["n_SEDOL_ROIC_Rank"]),
    n_SEDOL_ROIC_Rank_36MAgo_pct=lambda x: x["n_SEDOL"].div(
        x["n_SEDOL_ROIC_Rank_36MAgo"]
    ),
)
pd.options.display.precision = 2
display(g_roic_rank_shift[g_roic_rank_shift["ROIC_Rank"] == "rank1"].tail(50))


,date,ROIC_Rank,ROIC_Rank_36MAgo,n_SEDOL,n_SEDOL_ROIC_Rank,n_SEDOL_ROIC_Rank_36MAgo,n_SEDOL_ROIC_Rank_pct,n_SEDOL_ROIC_Rank_36MAgo_pct
4925,2025-01-31,rank1,rank1,126,208,225,0.61,0.56
4926,2025-01-31,rank1,rank2,42,208,210,0.20,0.20
4927,2025-01-31,rank1,rank3,18,208,217,0.09,0.08
4928,2025-01-31,rank1,rank4,10,208,206,0.05,0.05
4929,2025-01-31,rank1,rank5,12,208,188,0.06,0.06
4950,2025-02-28,rank1,rank1,126,209,224,0.60,0.56
4951,2025-02-28,rank1,rank2,42,209,210,0.20,0.20
4952,2025-02-28,rank1,rank3,18,209,218,0.09,0.08
4953,2025-02-28,rank1,rank4,10,209,205,0.05,0.05
4954,2025-02-28,rank1,rank5,13,209,191,0.06,0.07


### 4-8. Size Factor


#### 1. QoQ、YoY、3Yr CAGR, 5Y CAGR を計算してデータベースに保存する。


In [ ]:
factor_list = ["FF_SALES", "FF_ASSETS", "FF_COM_EQ", "FF_SHLDRS_EQ"]
period_list = ["QoQ", "YoY", "CAGR_3Y", "CAGR_5Y"]
factor_growth_list = [
    f"{factor}_{periods}"
    for factor, periods in itertools.product(factor_list, period_list)
]


In [ ]:
query = [f"SELECT * FROM `{table}`" for table in factor_list]
query = " UNION ALL ".join(query)

# ------------------------------------------------------
# load from database
# ------------------------------------------------------
with sqlite3.connect(financials_db_path) as conn:
    df_all = (
        pd.read_sql(query, con=conn, parse_dates=["date"])
        .sort_values("date", ignore_index=True)
        .assign(
            variable=lambda x: x["variable"].astype("category"),
            P_SYMBOL=lambda x: x["P_SYMBOL"].astype("category"),
        )
    ).sort_values(["variable", "P_SYMBOL", "date"], ignore_index=True)

display(df_all)
# groupby("variable")を使うことで、df_all全体を何度も走査(loc)するコストをゼロにする
# observed=True はcategory型を使う場合の高速化オプション
grouped = df_all.groupby("variable", observed=True)

# ------------------------------------------------------
# calculate growth and store to database
# ------------------------------------------------------
total_steps = len(factor_list)
for factor_name, df_factor in tqdm(grouped, total=total_steps, desc="Factors"):
    # df_factorはView(参照)の可能性があるため、計算用にコピーを作成
    # ここでメモリを食うが、factor単位なので全体コピーよりは軽い
    # かつ、ループの最後で解放される
    df_base = df_factor.copy()

    for growth in period_list:
        new_variable_name = f"{factor_name}_{growth}"
        df_result = roic_utils.calculate_growth(
            df=df_base, data_name=str(factor_name), growth_type=growth
        )

        # store to database
        db_utils.delete_table_from_database(
            db_path=financials_db_path, table_name=new_variable_name
        )
        factset_utils.store_to_database(
            df=df_result,
            db_path=financials_db_path,
            table_name=new_variable_name,
            verbose=False,
        )

    # ---- メモリ管理 ----
    # 一つのファクター処理が終わったら、使用した一時変数を削除してGC実行
    del df_base
    gc.collect()


,date,P_SYMBOL,value,variable
0,2005-08-31,0HSW-GB,2448.485005,FF_ASSETS
1,2005-09-30,0HSW-GB,2333.443621,FF_ASSETS
2,2005-10-31,0HSW-GB,2333.443621,FF_ASSETS
3,2005-11-30,0HSW-GB,2333.443621,FF_ASSETS
4,2005-12-30,0HSW-GB,NaN,FF_ASSETS
...,...,...,...,...
3165047,2025-06-30,ZURN-CH,24724.999387,FF_SHLDRS_EQ
3165048,2025-07-31,ZURN-CH,24724.999387,FF_SHLDRS_EQ
3165049,2025-08-29,ZURN-CH,24724.999387,FF_SHLDRS_EQ
3165050,2025-09-30,ZURN-CH,24724.999387,FF_SHLDRS_EQ


Factors: 100%|██████████| 4/4 [02:13<00:00, 33.38s/it]


#### 2. ファクターのランクを計算し、データベースに保存する。


In [ ]:
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))
factset_utils.process_rank_calculation_store_to_db(
    df_weight=df_weight,
    factor_list=factor_list,
    financials_db_path=financials_db_path,
    sector_neutral_mode=True,
)


🚀 処理開始: 4 件のタスク (Single Factor Mode)


Rank計算進捗: 100%|██████████| 4/4 [04:09<00:00, 62.43s/it] 


🎉 全てのランク計算・保存が完了しました


#### 3. データロード&欠損値確認：欠損値はセクター中央値で補完


In [ ]:
# ファクター計算のためのディスクリプターを指定
factor_list = [
    "FF_SALES_CAGR_3Y_PctRank_Sector_Neutral",
    "FF_ASSETS_PctRank_Sector_Neutral",
    "FF_COM_EQ_PctRank_Sector_Neutral",
    "FF_SHLDRS_EQ_PctRank_Sector_Neutral",
]

# -----------------------------------
# load data
# -----------------------------------

df_factor = factset_utils.load_financial_data(
    financials_db_path=financials_db_path, factor_list=factor_list
)

df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)
df = (
    factset_utils.merge_idx_constituents_and_financials(
        df_weight=df_weight, df_factor=df_factor
    )
    .fillna(np.nan)
    .dropna(subset=factor_list, how="all")
    .dropna(subset=["Weight (%)"], ignore_index=True)
)
display(df.head())

df = factset_utils.check_missing_value_and_fill_by_sector_median(
    df=df, factor_list=factor_list
)


,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),FF_ASSETS_PctRank_Sector_Neutral,FF_COM_EQ_PctRank_Sector_Neutral,FF_SALES_CAGR_3Y_PctRank_Sector_Neutral,FF_SHLDRS_EQ_PctRank_Sector_Neutral
0,2005-08-31,0HSW-GB,3335442,UKIGDP1,Telent PLC,Information Technology,Technology Hardware & Equipment,0.005496,0.421053,0.270677,NaN,0.270677
1,2005-08-31,0II3.XX1-GB,0299303,UKIBKB1,Emap Plc,Consumer Discretionary,Media,0.019501,0.240310,0.128906,NaN,0.128906
2,2005-08-31,0MDJ-GB,0610700,UKIBBF1,Cadbury PLC,Consumer Staples,Food Beverage & Tobacco,0.106288,0.765957,0.621053,NaN,0.621053
3,2005-08-31,0N1N-GB,0230346,UKIBGE1,Arriva Plc Ord,Industrials,Transportation,0.010309,0.331754,0.273585,NaN,0.268868
4,2005-08-31,0N3I-GB,0896265,UKIDBM1,Tomkins PLC,Industrials,Capital Goods,0.020285,0.421801,0.344340,NaN,0.344340


📋 欠損値の状況（補完前）
FF_SALES_CAGR_3Y_PctRank_Sector_Neutral      : 59,070件 ( 18.5%)
FF_ASSETS_PctRank_Sector_Neutral             :  1,966件 (  0.6%)
FF_COM_EQ_PctRank_Sector_Neutral             :  1,756件 (  0.6%)
FF_SHLDRS_EQ_PctRank_Sector_Neutral          :  1,756件 (  0.6%)

⏳ セクター中央値で補完中...

📋 欠損値の状況（セクター中央値補完後）
⚠️ FF_SALES_CAGR_3Y_PctRank_Sector_Neutral      : 52,190件 ( 16.4%) | 補完: 6,880件
✅ FF_ASSETS_PctRank_Sector_Neutral             :      0件 (  0.0%) | 補完: 1,966件
✅ FF_COM_EQ_PctRank_Sector_Neutral             :      0件 (  0.0%) | 補完: 1,756件
✅ FF_SHLDRS_EQ_PctRank_Sector_Neutral          :      0件 (  0.0%) | 補完: 1,756件

⚠️  セクター補完で埋まらない欠損値を全体中央値で再補完...
------------------------------------------------------------
⚠️  FF_SALES_CAGR_3Y_PctRank_Sector_Neutral: 全体でも欠損 → 0.5（中立値）で補完
------------------------------------------------------------

✅ 最終欠損値チェック
✅ FF_SALES_CAGR_3Y_PctRank_Sector_Neutral      : 欠損なし
✅ FF_ASSETS_PctRank_Sector_Neutral             : 欠損なし
✅ FF_COM_EQ_PctRank_Sector_Neu

#### 4. ファクター計算


In [ ]:
# -----------------------------------
# Factorの計算
# -----------------------------------
# ウェイト設定
blend_weight_1 = {
    "FF_SALES_CAGR_3Y_PctRank_Sector_Neutral": 0.25,
    "FF_ASSETS_PctRank_Sector_Neutral": 0.25,
    "FF_COM_EQ_PctRank_Sector_Neutral": 0.25,
    "FF_SHLDRS_EQ_PctRank_Sector_Neutral": 0.25,
}
blend_weight_2 = {
    "FF_ASSETS_PctRank_Sector_Neutral": 1 / 3,
    "FF_COM_EQ_PctRank_Sector_Neutral": 1 / 3,
    "FF_SHLDRS_EQ_PctRank_Sector_Neutral": 1 / 3,
}
factor_name = "Factor_Size_ex_Sales"

df = factset_utils.create_factor(
    df=df, factor_name=factor_name, blend_weight=blend_weight_2
)
display(df.tail(5))

# -----------------------------------
# Store to database
# -----------------------------------
for variable in [f"{factor_name}_Score", f"{factor_name}_Score_Rank"]:
    df_slice = (
        df[["date", "P_SYMBOL", variable]]
        .assign(variable=variable, date=lambda x: pd.to_datetime(x["date"]))
        .rename(columns={variable: "value"})
    )
    db_utils.delete_table_from_database(db_path=financials_db_path, table_name=variable)
    factset_utils.store_to_database(
        df=df_slice,
        db_path=financials_db_path,
        table_name=variable,
        verbose=True,
    )



📊 Factor 計算
ウェイト設定:
  FF_ASSETS_PctRank_Sector_Neutral             : 33.3%
  FF_COM_EQ_PctRank_Sector_Neutral             : 33.3%
  FF_SHLDRS_EQ_PctRank_Sector_Neutral          : 33.3%

✅ Score 計算完了
   平均: 0.5040
   標準偏差: 0.2778
   最小値: 0.0067
   最大値: 1.0000

📊 Rank 分布:
  rank5: 63,924件 ( 20.0%)
  rank4: 63,744件 ( 20.0%)
  rank3: 63,758件 ( 20.0%)
  rank2: 63,730件 ( 20.0%)
  rank1: 63,814件 ( 20.0%)

🎯 最終データサンプル:
             date P_SYMBOL    SEDOL             GICS Sector  \
318950 2025-10-31   WTB-GB  B1KJJ40  Consumer Discretionary   
318951 2025-10-31   WTC-AU  BZ8GX83  Information Technology   
318952 2025-10-31  WTRG-US  BLCF3J9               Utilities   
318953 2025-10-31   WTW-US  BDB6Q21              Financials   
318954 2025-10-31    WY-US  2958936             Real Estate   
318955 2025-10-31     X-CA  B8KH5G7              Financials   
318956 2025-10-31   XEL-US  2614807               Utilities   
318957 2025-10-31   XOM-US  2326618                  Energy   
318958 2025-10-31

,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),FF_ASSETS_PctRank_Sector_Neutral,FF_COM_EQ_PctRank_Sector_Neutral,FF_SALES_CAGR_3Y_PctRank_Sector_Neutral,FF_SHLDRS_EQ_PctRank_Sector_Neutral,Factor_Size_ex_Sales_Score,Factor_Size_ex_Sales_Score_Rank
318965,2025-10-31,ZAL-DE,BQV0SV7,GERZQZ1,ザランド,Consumer Discretionary,Consumer Discretionary Distribution & Retail,0.008099,0.282353,0.364706,0.755814,0.364706,0.337255,rank4
318966,2025-10-31,ZBH-US,2783815,USA4JT1,ジンマー・バイオメット・ホールディングス,Health Care,Health Care Equipment & Services,0.025627,0.534653,0.633663,0.438776,0.633663,0.600660,rank2
318967,2025-10-31,ZBRA-US,2989356,USAP8H1,ゼブラ・テクノロジーズ・コーポレーション Class A,Information Technology,Technology Hardware & Equipment,0.017637,0.349398,0.397590,0.190476,0.397590,0.381526,rank4
318968,2025-10-31,ZTS-US,B95WG16,USBANZ1,ゾエティス Class A,Health Care,Pharmaceuticals Biotechnology & Life Sciences,0.082630,0.415842,0.376238,0.459184,0.376238,0.389439,rank3
318969,2025-10-31,ZURN-CH,5983816,SWIAFM2,チューリッヒ・インシュアランス・グループ,Financials,Insurance,0.131275,0.697436,0.641026,0.952632,0.630769,0.656410,rank2


既存の 0 行との重複をチェックしました。318947 行を新たに追加します。
  -> Factor_Size_ex_Sales_Score: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。318947 行を新たに追加します。
  -> Factor_Size_ex_Sales_Score_Rank: データの書き込みが完了しました。


### 4-9. Leverage Factor


- FF_DEBT_EQ(Debt to Equity ratio)
- FF_LIABS_SHLDRS_EQ(Total Liabilities to Shareholders' Equity ratio)
- FF_NET_DEBT(Net Debt)


#### (🚧 実装予定)📝 ディスクリプター追加：NET_DEBT_TO_EBITDA_OPER

-> FF_NET_DEBT / FF_EBITDA_OPER


In [ ]:
descriptor_list = ["FF_NET_DEBT", "FF_EBITDA_OPER"]
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))
# ファクター
df_factor = factset_utils.load_financial_data(
    financials_db_path=financials_db_path, factor_list=descriptor_list
).assign(date=lambda x: pd.to_datetime(x["date"]))
df_merged = factset_utils.merge_idx_constituents_and_financials(
    df_weight=df_weight, df_factor=df_factor
)

df_merged["NET_DEBT_TO_EBITDA_OPER"] = df_merged["FF_NET_DEBT"].div(
    df_merged["FF_EBITDA_OPER"]
)
display(df_merged.loc[df_merged["NET_DEBT_TO_EBITDA_OPER"] < 0])


,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),FF_EBITDA_OPER,FF_NET_DEBT,NET_DEBT_TO_EBITDA_OPER
78939,2005-08-31,0UAN-GB,0128269,UKIAYK1,Invesco Ltd.,Financials,Financial Services,0.025020,156.54217,-320.523897,-2.047524
78956,2005-08-31,2878-HK,B00G568,HKGCXQ1,Solomon Systech (International) Ltd.,Information Technology,Semiconductors & Semiconductor Equipment,0.002918,45.57000,-152.981004,-3.357055
78960,2005-08-31,315-HK,6856995,HKGBCL1,SmarTone Telecommunications Holdings Limited,Communication Services,Telecommunication Services,0.001761,50.32445,-148.727957,-2.955382
78963,2005-08-31,330-HK,6321642,HKGAXW1,Esprit Holdings Limited,Consumer Discretionary,Consumer Discretionary Distribution & Retail,0.036683,283.80326,-222.383163,-0.783582
78966,2005-08-31,494-HK,6286257,HKGAJG1,Li & Fung Limited,Consumer Discretionary,Consumer Discretionary Distribution & Retail,0.018461,76.74025,-18.533442,-0.241509
...,...,...,...,...,...,...,...,...,...,...,...
400839,2025-10-31,WST-US,2950482,USAOV41,ウエスト・ファーマシューティカル・サービシーズ,Health Care,Pharmaceuticals Biotechnology & Life Sciences,0.026103,216.70000,-325.500000,-1.502077
400841,2025-10-31,WTC-AU,BZ8GX83,AUSIDZ1,ワイズテック・グローバル,Information Technology,Software & Services,0.011702,218.60000,-55.700003,-0.254803
400848,2025-10-31,XRO-AU,B8P4LP4,AUSHTH1,ゼロ,Information Technology,Software & Services,0.020128,208.45848,-2004.614013,-9.616371
400850,2025-10-31,XYZ-US,BYNZGK1,USBDBN1,"Block, Inc. Class A",Financials,Financial Services,0.054288,865.35900,-1336.948000,-1.544963


#### 1. ファクター値を計算してデータベースに保存する


In [ ]:
factor_list = ["FF_DEBT_EQ", "FF_LIABS_SHLDRS_EQ", "FF_NET_DEBT"]


In [ ]:
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))

factset_utils.process_rank_calculation_store_to_db(
    df_weight=df_weight, factor_list=factor_list, financials_db_path=financials_db_path
)


[autoreload of src.factset_utils failed: Traceback (most recent call last):
  File "c:\Users\Yuki Hata\Desktop\papers\.venv\Lib\site-packages\IPython\extensions\autoreload.py", line 322, in check
    elif self.deduper_reloader.maybe_reload_module(m):
         ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
  File "c:\Users\Yuki Hata\Desktop\papers\.venv\Lib\site-packages\IPython\extensions\deduperreload\deduperreload.py", line 545, in maybe_reload_module
    new_source_code = f.read()
UnicodeDecodeError: 'cp932' codec can't decode byte 0x87 in position 725: illegal multibyte sequence
]


🚀 処理開始: 3 件のタスク


Rank計算進捗: 100%|██████████| 3/3 [00:32<00:00, 10.69s/it]


🎉 全てのランク計算・保存が完了しました


### 2. データロード&欠損値確認: 欠損値はセクター中央値で補完


In [ ]:
# ファクター計算のためのディスクリプターを指定
factor_list = [
    "FF_DEBT_EQ_Inv_PctRank",
    "FF_LIABS_SHLDRS_EQ_Inv_PctRank",
    "FF_NET_DEBT_Inv_PctRank",
]

# -----------------------------------
# load data
# -----------------------------------

df_factor = factset_utils.load_financial_data(
    financials_db_path=financials_db_path, factor_list=factor_list
)

df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)
df = (
    factset_utils.merge_idx_constituents_and_financials(
        df_weight=df_weight, df_factor=df_factor
    )
    .fillna(np.nan)
    .dropna(subset=factor_list, how="all")
    .dropna(subset=["Weight (%)"], ignore_index=True)
)
display(df.head())

df = factset_utils.check_missing_value_and_fill_by_sector_median(
    df=df, factor_list=factor_list
)


,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),FF_DEBT_EQ_Inv_PctRank,FF_LIABS_SHLDRS_EQ_Inv_PctRank,FF_NET_DEBT_Inv_PctRank
0,2005-08-31,0HSW-GB,3335442,UKIGDP1,Telent PLC,Information Technology,Technology Hardware & Equipment,0.005496,0.542636,0.578947,NaN
1,2005-08-31,0II3.XX1-GB,0299303,UKIBKB1,Emap Plc,Consumer Discretionary,Media,0.019501,0.197531,0.758755,NaN
2,2005-08-31,0MDJ-GB,0610700,UKIBBF1,Cadbury PLC,Consumer Staples,Food Beverage & Tobacco,0.106288,0.108696,0.234043,0.129870
3,2005-08-31,0N1N-GB,0230346,UKIBGE1,Arriva Plc Ord,Industrials,Transportation,0.010309,0.323671,0.671429,0.564246
4,2005-08-31,0N3I-GB,0896265,UKIDBM1,Tomkins PLC,Industrials,Capital Goods,0.020285,0.328502,0.580952,0.519553


📋 欠損値の状況（補完前）
FF_DEBT_EQ_Inv_PctRank                       :  7,872件 (  2.5%)
FF_LIABS_SHLDRS_EQ_Inv_PctRank               :    300件 (  0.1%)
FF_NET_DEBT_Inv_PctRank                      : 18,313件 (  5.8%)

⏳ セクター中央値で補完中...

📋 欠損値の状況（セクター中央値補完後）
✅ FF_DEBT_EQ_Inv_PctRank                       :      0件 (  0.0%) | 補完: 7,872件
✅ FF_LIABS_SHLDRS_EQ_Inv_PctRank               :      0件 (  0.0%) | 補完: 300件
✅ FF_NET_DEBT_Inv_PctRank                      :      0件 (  0.0%) | 補完: 18,313件

✅ 最終欠損値チェック
✅ FF_DEBT_EQ_Inv_PctRank                       : 欠損なし
✅ FF_LIABS_SHLDRS_EQ_Inv_PctRank               : 欠損なし
✅ FF_NET_DEBT_Inv_PctRank                      : 欠損なし
🎉 すべての欠損値が補完されました！


### 3. ファクター計算


In [ ]:
# -----------------------------------
# Factorの計算
# -----------------------------------
# ウェイト設定
blend_weight = {
    "FF_DEBT_EQ_Inv_PctRank": 1 / 3,
    "FF_LIABS_SHLDRS_EQ_Inv_PctRank": 1 / 3,
    "FF_NET_DEBT_Inv_PctRank": 1 / 3,
}
factor_name = "Factor_Leverage"

df = factset_utils.create_factor(
    df=df, factor_name=factor_name, blend_weight=blend_weight
)
display(df.tail(5))

# -----------------------------------
# Store to database
# -----------------------------------
for variable in [f"{factor_name}_Score", f"{factor_name}_Score_Rank"]:
    df_slice = (
        df[["date", "P_SYMBOL", variable]]
        .assign(variable=variable, date=lambda x: pd.to_datetime(x["date"]))
        .rename(columns={variable: "value"})
    )
    db_utils.delete_table_from_database(db_path=financials_db_path, table_name=variable)
    factset_utils.store_to_database(
        df=df_slice,
        db_path=financials_db_path,
        table_name=variable,
        verbose=True,
    )



📊 Factor 計算
ウェイト設定:
  FF_DEBT_EQ_Inv_PctRank                       : 33.3%
  FF_LIABS_SHLDRS_EQ_Inv_PctRank               : 33.3%
  FF_NET_DEBT_Inv_PctRank                      : 33.3%

✅ Score 計算完了
   平均: 0.4959
   標準偏差: 0.2305
   最小値: 0.0000
   最大値: 0.9831

📊 Rank 分布:
  rank5: 63,534件 ( 20.0%)
  rank4: 63,382件 ( 20.0%)
  rank3: 63,389件 ( 20.0%)
  rank2: 63,377件 ( 20.0%)
  rank1: 63,478件 ( 20.0%)

🎯 最終データサンプル:
             date P_SYMBOL    SEDOL             GICS Sector  \
317140 2025-10-31   WTB-GB  B1KJJ40  Consumer Discretionary   
317141 2025-10-31   WTC-AU  BZ8GX83  Information Technology   
317142 2025-10-31  WTRG-US  BLCF3J9               Utilities   
317143 2025-10-31   WTW-US  BDB6Q21              Financials   
317144 2025-10-31    WY-US  2958936             Real Estate   
317145 2025-10-31     X-CA  B8KH5G7              Financials   
317146 2025-10-31   XEL-US  2614807               Utilities   
317147 2025-10-31   XOM-US  2326618                  Energy   
317148 2025-10-31

,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),FF_DEBT_EQ_Inv_PctRank,FF_LIABS_SHLDRS_EQ_Inv_PctRank,FF_NET_DEBT_Inv_PctRank,Factor_Leverage_Score,Factor_Leverage_Score_Rank
317155,2025-10-31,ZAL-DE,BQV0SV7,GERZQZ1,ザランド,Consumer Discretionary,Consumer Discretionary Distribution & Retail,0.008099,0.689189,0.717647,0.788235,0.731691,rank1
317156,2025-10-31,ZBH-US,2783815,USA4JT1,ジンマー・バイオメット・ホールディングス,Health Care,Health Care Equipment & Services,0.025627,0.536842,0.465347,0.396040,0.466076,rank3
317157,2025-10-31,ZBRA-US,2989356,USAP8H1,ゼブラ・テクノロジーズ・コーポレーション Class A,Information Technology,Technology Hardware & Equipment,0.017637,0.362500,0.650602,0.421687,0.478263,rank3
317158,2025-10-31,ZTS-US,B95WG16,USBANZ1,ゾエティス Class A,Health Care,Pharmaceuticals Biotechnology & Life Sciences,0.082630,0.136842,0.584158,0.465347,0.395449,rank4
317159,2025-10-31,ZURN-CH,5983816,SWIAFM2,チューリッヒ・インシュアランス・グループ,Financials,Insurance,0.131275,0.583333,0.304124,0.371134,0.419530,rank4


既存の 0 行との重複をチェックしました。317137 行を新たに追加します。
  -> Factor_Leverage_Score: データの書き込みが完了しました。
既存の 0 行との重複をチェックしました。317137 行を新たに追加します。
  -> Factor_Leverage_Score_Rank: データの書き込みが完了しました。


### 4-10. Momentum/Reversal Factor


#### 1. 区間リターンを計算


In [ ]:
# ----------------------------------------
# load price data
# ----------------------------------------
df_price = (
    price.load_fg_price(db_path=financials_db_path)
    .reset_index()
    .astype({"P_SYMBOL": "category"})
    .assign(date=lambda x: pd.to_datetime(x["date"]))
).sort_values(["date", "P_SYMBOL"], ignore_index=True)

# ----------------------------------------
# 区間リターンを計算
# ----------------------------------------

df_calculated = roic_utils.calculate_interval_returns(df=df_price).drop(
    columns=["FG_PRICE"]
)
target_columns = [c for c in df_calculated.columns if c not in ["date", "P_SYMBOL"]]

# store to database
for col_name in target_columns:
    df_slice = df_calculated[["date", "P_SYMBOL", col_name]].copy()
    df_slice = (
        df_slice.rename(columns={col_name: "value"})
        .assign(variable=col_name)
        .reindex(columns=["date", "P_SYMBOL", "variable", "value"])
    )
    df_slice = df_slice.dropna(subset=["value"])
    factset_utils.store_to_database(
        df=df_slice, db_path=financials_db_path, table_name=col_name
    )


テーブル 'Return_1YAgo_to_Current' に追加すべき新しいデータはありませんでした。スキップします。
テーブル 'Return_1YAgo_to_1MAgo' に追加すべき新しいデータはありませんでした。スキップします。
テーブル 'Return_2YAgo_to_1YAgo' に追加すべき新しいデータはありませんでした。スキップします。
テーブル 'Return_3YAgo_to_2YAgo' に追加すべき新しいデータはありませんでした。スキップします。
テーブル 'Return_3YAgo_to_1YAgo' に追加すべき新しいデータはありませんでした。スキップします。
テーブル 'Return_1MAgo_to_Current' に追加すべき新しいデータはありませんでした。スキップします。


#### 2. ファクター値を計算して DB 保存


In [ ]:
# inverse = False
momentum_descriptor_list = [
    "Return_1YAgo_to_Current",
    "Return_1YAgo_to_1MAgo",
    "Return_2YAgo_to_1YAgo",
    "Return_3YAgo_to_2YAgo",
    "Return_3YAgo_to_1YAgo",
]

# inverse = True
reversal_descriptor_list = [
    "Return_3YAgo_to_1YAgo",
    "Return_1MAgo_to_Current",
]


In [ ]:
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))

# -----------------------------------
# Momentum -> inversed=False
# Reversal -> inversed=True
# セクター中立あり・なし両方
# -----------------------------------
# for factor_list, inversed in [
#     (momentum_descriptor_list, False),
#     (reversal_descriptor_list, True),
# ]:
#     for sector_neutral in [False, True]:
#         factset_utils.process_rank_calculation_store_to_db(
#             df_weight=df_weight,
#             factor_list=factor_list,
#             financials_db_path=financials_db_path,
#             sector_neutral_mode=sector_neutral,
#             inversed=inversed,
#         )

for sector_neutral in [True, False]:
    factset_utils.process_rank_calculation_store_to_db(
        df_weight=df_weight,
        factor_list=momentum_descriptor_list,
        financials_db_path=financials_db_path,
        sector_neutral_mode=sector_neutral,
        inversed=False,
    )


🚀 処理開始: 5 件のタスク (Single Factor Mode)


Rank計算進捗: 100%|██████████| 5/5 [03:58<00:00, 47.64s/it]


🎉 全てのランク計算・保存が完了しました
🚀 処理開始: 5 件のタスク (Single Factor Mode)


Rank計算進捗: 100%|██████████| 5/5 [04:18<00:00, 51.75s/it] 


🎉 全てのランク計算・保存が完了しました


#### 3. データロード&欠損値確認: 欠損値はセクター中央値で補完


In [ ]:
# ファクター計算のためのディスクリプターを指定
factor_list = [s + "_PctRank" for s in momentum_descriptor_list]

# -----------------------------------
# load data
# -----------------------------------

df_factor = factset_utils.load_financial_data(
    financials_db_path=financials_db_path, factor_list=factor_list
)

df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
)
df = (
    factset_utils.merge_idx_constituents_and_financials(
        df_weight=df_weight, df_factor=df_factor
    )
    .fillna(np.nan)
    .dropna(subset=factor_list, how="all")
    .dropna(subset=["Weight (%)"], ignore_index=True)
)
df = df.loc[df["date"].dt.year >= 2009]
display(df.head())

df = factset_utils.check_missing_value_and_fill_by_sector_median(
    df=df, factor_list=factor_list
)


,date,P_SYMBOL,SEDOL,Asset ID,FG_COMPANY_NAME,GICS Sector,GICS Industry Group,Weight (%),Return_1YAgo_to_1MAgo_PctRank,Return_1YAgo_to_Current_PctRank,Return_2YAgo_to_1YAgo_PctRank,Return_3YAgo_to_2YAgo_PctRank
30834,2009-03-31,0MDJ-GB,B2PF6M7,UKIBBF1,Cadbury PLC,Consumer Staples,Food Beverage & Tobacco,0.078896,0.737864,0.747573,0.247525,0.356436
30835,2009-03-31,0N3I-GB,0896265,UKIDBM1,Tomkins PLC,Industrials,Capital Goods,0.011803,0.722772,0.742574,0.071066,0.031250
30836,2009-03-31,0P7J-GB,0028262,UKIAPY1,Amec Foster Wheeler plc,Energy,Energy,0.019520,0.854545,0.790909,0.688073,0.924528
30837,2009-03-31,1-HK,6190273,HKGAAE1,CKハチソン・ホールディングス,Financials,Real Estate,0.091940,0.711191,0.689531,0.923358,0.687500
30838,2009-03-31,10-HK,6408352,HKGAGG1,Hang Lung Group Limited,Financials,Real Estate,0.020283,0.696751,0.736462,0.985401,0.959559


📋 欠損値の状況（補完前）
Return_1YAgo_to_Current_PctRank              :      6件 (  0.0%)
Return_1YAgo_to_1MAgo_PctRank                :      0件 (  0.0%)
Return_2YAgo_to_1YAgo_PctRank                :  2,146件 (  1.2%)
Return_3YAgo_to_2YAgo_PctRank                :  4,703件 (  2.6%)

⏳ セクター中央値で補完中...

📋 欠損値の状況（セクター中央値補完後）
✅ Return_1YAgo_to_Current_PctRank              :      0件 (  0.0%) | 補完: 6件
✅ Return_1YAgo_to_1MAgo_PctRank                :      0件 (  0.0%) | 補完: 0件
✅ Return_2YAgo_to_1YAgo_PctRank                :      0件 (  0.0%) | 補完: 2,146件
✅ Return_3YAgo_to_2YAgo_PctRank                :      0件 (  0.0%) | 補完: 4,703件

✅ 最終欠損値チェック
✅ Return_1YAgo_to_Current_PctRank              : 欠損なし
✅ Return_1YAgo_to_1MAgo_PctRank                : 欠損なし
✅ Return_2YAgo_to_1YAgo_PctRank                : 欠損なし
✅ Return_3YAgo_to_2YAgo_PctRank                : 欠損なし
🎉 すべての欠損値が補完されました！


#### （🚧 工事中）4. ファクター計算


### 4-11. その他の財務項目

時点でのランク、パーセントランク、ZScore のみ計算


In [ ]:
# ファクター値
factor_list = [
    # "FF_ASSETS",
    # "FF_BPS",
    # "FF_BPS_TANG",
    # "FF_CAPEX",
    # "FF_CASH_ST",
    # "FF_COGS",
    # "FF_COM_EQ",
    # "FF_CURR_RATIO",
    # "FF_DEBT",
    # "FF_DEBT_ENTRPR_VAL",
    # "FF_DEBT_EQ",
    # "FF_DEBT_LT",
    # "FF_DEBT_ST",
    # "FF_DEP_AMORT_EXP",
    # "FF_DIV_YLD",
    # "FF_DPS",
    # "FF_EBITDA_OPER",
    # "FF_EBITDA_OPER_MGN",
    # "FF_EBIT_OPER",
    # "FF_EBIT_OPER_MGN",
    # "FF_ENTRPR_VAL_EBITDA_OPER",
    # "FF_ENTRPR_VAL_EBIT_OPER",
    # "FF_ENTRPR_VAL_SALES",
    # "FF_EPS",
    # "FF_EPS_DIL",
    # "FF_FREE_CF",
    # "FF_FREE_PS_CF",
    # "FF_GROSS_INC",
    # "FF_GROSS_MGN",
    # "FF_INC_TAX",
    # "FF_INT_EXP_NET",
    # "FF_LIABS",
    # "FF_LIABS_SHLDRS_EQ",
    # "FF_MIN_INT_ACCUM",
    # "FF_NET_DEBT",
    # "FF_NET_INC",
    # "FF_NET_MGN",
    # "FF_OPER_CF",
    # "FF_OPER_INC",
    # "FF_OPER_MGN",
    # "FF_OPER_PS_NET_CF",
    # "FF_PAY_OUT_RATIO",
    # "FF_PBK",
    # "FF_PE",
    # "FF_PFD_STK",
    # "FF_PPE_NET",
    # "FF_PSALES",
    # "FF_PTX_INC",
    # "FF_PTX_MGN",
    # "FF_QUICK_RATIO",
    "FF_ROA",
    "FF_ROE",
    "FF_ROIC",
    "FF_ROTC",
    "FF_SALES",
    "FF_SALES_PS",
    "FF_SGA",
    # "FF_SHLDRS_EQ",
    # "FF_STK_OPT_EXP",
    # "FF_STK_PURCH_CF",
    # "FF_TAX_RATE",
    # "FF_WKCAP",
]


In [ ]:
# -----------------------------------
# load data
# -----------------------------------
# 構成銘柄情報
df_weight = factset_utils.load_index_constituents(
    factset_index_db_path=factset_index_db_path, UNIVERSE_CODE=UNIVERSE_CODE
).assign(date=lambda x: pd.to_datetime(x["date"]))

for sector_neutral_mode in [True, False]:
    factset_utils.process_rank_calculation_store_to_db(
        df_weight=df_weight,
        factor_list=factor_list,
        financials_db_path=financials_db_path,
        sector_neutral_mode=sector_neutral_mode,
    )


🚀 処理開始: 7 件のタスク (Single Factor Mode)


Rank計算進捗: 100%|██████████| 7/7 [05:39<00:00, 48.51s/it]


🎉 全てのランク計算・保存が完了しました
🚀 処理開始: 7 件のタスク (Single Factor Mode)


Rank計算進捗: 100%|██████████| 7/7 [05:22<00:00, 46.01s/it]


🎉 全てのランク計算・保存が完了しました


### ✅ データベース内容確認


In [ ]:
table_names = sorted(db_utils.get_table_names(db_path=financials_db_path))
print(f"全{len(table_names)}テーブル")
display(table_names)

with sqlite3.connect(factset_index_db_path) as conn:
    df = pd.read_sql(
        f"SELECT * FROM {UNIVERSE_CODE} LIMIT 5", parse_dates=["date"], con=conn
    )
    display(df)
    display(df.columns)


全1256テーブル


['Active_Return_12M',
 'Active_Return_12M_annlzd',
 'Active_Return_1M',
 'Active_Return_1M_annlzd',
 'Active_Return_3M',
 'Active_Return_3M_annlzd',
 'Active_Return_3Y',
 'Active_Return_3Y_annlzd',
 'Active_Return_5Y',
 'Active_Return_5Y_annlzd',
 'Active_Return_6M',
 'Active_Return_6M_annlzd',
 'FF_ASSETS',
 'FF_ASSETS_CAGR_3Y',
 'FF_ASSETS_CAGR_3Y_PctRank',
 'FF_ASSETS_CAGR_3Y_PctRank_Sector_Neutral',
 'FF_ASSETS_CAGR_3Y_Rank',
 'FF_ASSETS_CAGR_3Y_Rank_Sector_Neutral',
 'FF_ASSETS_CAGR_3Y_ZScore',
 'FF_ASSETS_CAGR_3Y_ZScore_Sector_Neutral',
 'FF_ASSETS_CAGR_5Y',
 'FF_ASSETS_CAGR_5Y_PctRank',
 'FF_ASSETS_CAGR_5Y_PctRank_Sector_Neutral',
 'FF_ASSETS_CAGR_5Y_Rank',
 'FF_ASSETS_CAGR_5Y_Rank_Sector_Neutral',
 'FF_ASSETS_CAGR_5Y_ZScore',
 'FF_ASSETS_CAGR_5Y_ZScore_Sector_Neutral',
 'FF_ASSETS_PctRank',
 'FF_ASSETS_QoQ',
 'FF_ASSETS_QoQ_PctRank',
 'FF_ASSETS_QoQ_PctRank_Sector_Neutral',
 'FF_ASSETS_QoQ_Rank',
 'FF_ASSETS_QoQ_Rank_Sector_Neutral',
 'FF_ASSETS_QoQ_ZScore',
 'FF_ASSETS_QoQ_ZSc

,Universe,Universe_code_BPM,date,Name,Bloomberg Ticker,BloombergID,Asset ID,Asset ID Type,SEDOL,Country,...,FG_COMPANY_NAME_CUSIP,P_SYMBOL_CUSIP,ISIN,FG_COMPANY_NAME_ISIN,P_SYMBOL_ISIN,CODE_JP,FG_COMPANY_NAME_CODE_JP,P_SYMBOL_CODE_JP,P_SYMBOL,FG_COMPANY_NAME
0,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,21ST CENTURY FOX,None,None,AUSBIN2,BARRAID,662075,AUS,...,None,None,None,None,None,None,None,None,FOXLV-AU,Twenty-First Century Fox Inc. Class A CDI
1,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,21ST CENTURY FOX,None,None,AUSBIN1,BARRAID,688692,AUS,...,None,None,None,None,None,None,None,None,FOX-AU,Twenty-First Century Fox Inc. Class B CDI
2,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,3I GROUP PLC,None,None,UKIENL1,BARRAID,0888693,GBR,...,スリーアイ・グループ,TGOPF-US,GB0008886938,スリーアイ・グループ,III-GB,None,None,None,III-GB,スリーアイ・グループ
3,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,3M CO,None,None,USAJ8P1,BARRAID,2595708,USA,...,3Mカンパニー,MMM-US,US6040591058,3Mカンパニー,MMM-US,None,None,None,MMM-US,3Mカンパニー
4,MSCI KOKUSAI - Daily,MSXJPN_AD,2000-01-31,ABB LTD,None,None,SWIAAN1,BARRAID,5661190,CHE,...,ABB,None,CH0003846620,ABB,ABBN-CH,None,None,None,ABBN-CH,ABB


Index(['Universe', 'Universe_code_BPM', 'date', 'Name', 'Bloomberg Ticker',
       'BloombergID', 'Asset ID', 'Asset ID Type', 'SEDOL', 'Country',
       'GICS Sector', 'GICS Industry', 'GICS Industry Group',
       'GICS Sub-Industry', 'Holdings', 'Weight (%)', 'Mkt Value',
       'FG_COMPANY_NAME_SEDOL', 'P_SYMBOL_SEDOL', 'CUSIP',
       'FG_COMPANY_NAME_CUSIP', 'P_SYMBOL_CUSIP', 'ISIN',
       'FG_COMPANY_NAME_ISIN', 'P_SYMBOL_ISIN', 'CODE_JP',
       'FG_COMPANY_NAME_CODE_JP', 'P_SYMBOL_CODE_JP', 'P_SYMBOL',
       'FG_COMPANY_NAME'],
      dtype='object')

## 5. 欠損確認


### ✅ データベース確認


In [ ]:
table_names = sorted(db_utils.get_table_names(db_path=financials_db_path))
print(f"全{len(table_names)}テーブル")
display(table_names)

with sqlite3.connect(factset_index_db_path) as conn:
    df_weight = pd.read_sql(
        f"SELECT * FROM {UNIVERSE_CODE}", parse_dates=["date"], con=conn
    )
    display(df_weight.columns)
    display(df_weight.tail(5))


全454テーブル


['Active_Return_1M',
 'Active_Return_1M_annlzd',
 'Active_Return_3M',
 'Active_Return_3M_annlzd',
 'Active_Return_3Y',
 'Active_Return_3Y_annlzd',
 'Active_Return_5Y',
 'Active_Return_5Y_annlzd',
 'Active_Return_6M',
 'Active_Return_6M_annlzd',
 'FF_ASSETS',
 'FF_ASSETS_CAGR_3Y',
 'FF_ASSETS_CAGR_3Y_PctRank',
 'FF_ASSETS_CAGR_3Y_Rank',
 'FF_ASSETS_CAGR_3Y_ZScore',
 'FF_ASSETS_CAGR_5Y',
 'FF_ASSETS_CAGR_5Y_PctRank',
 'FF_ASSETS_CAGR_5Y_Rank',
 'FF_ASSETS_CAGR_5Y_ZScore',
 'FF_ASSETS_PctRank',
 'FF_ASSETS_QoQ',
 'FF_ASSETS_QoQ_PctRank',
 'FF_ASSETS_QoQ_Rank',
 'FF_ASSETS_QoQ_ZScore',
 'FF_ASSETS_Rank',
 'FF_ASSETS_YoY',
 'FF_ASSETS_YoY_PctRank',
 'FF_ASSETS_YoY_Rank',
 'FF_ASSETS_YoY_ZScore',
 'FF_ASSETS_ZScore',
 'FF_BPS',
 'FF_BPS_PctRank',
 'FF_BPS_Rank',
 'FF_BPS_TANG',
 'FF_BPS_TANG_PctRank',
 'FF_BPS_TANG_Rank',
 'FF_BPS_TANG_ZScore',
 'FF_BPS_ZScore',
 'FF_CAPEX',
 'FF_CAPEX_PctRank',
 'FF_CAPEX_Rank',
 'FF_CAPEX_ZScore',
 'FF_CASH_ST',
 'FF_CASH_ST_PctRank',
 'FF_CASH_ST_Rank',
 

Index(['Universe', 'Universe_code_BPM', 'date', 'Name', 'Bloomberg Ticker',
       'BloombergID', 'Asset ID', 'Asset ID Type', 'SEDOL', 'Country',
       'GICS Sector', 'GICS Industry', 'GICS Industry Group',
       'GICS Sub-Industry', 'Holdings', 'Weight (%)', 'Mkt Value',
       'FG_COMPANY_NAME_SEDOL', 'P_SYMBOL_SEDOL', 'CUSIP',
       'FG_COMPANY_NAME_CUSIP', 'P_SYMBOL_CUSIP', 'ISIN',
       'FG_COMPANY_NAME_ISIN', 'P_SYMBOL_ISIN', 'CODE_JP',
       'FG_COMPANY_NAME_CODE_JP', 'P_SYMBOL_CODE_JP', 'P_SYMBOL',
       'FG_COMPANY_NAME'],
      dtype='object')

,Universe,Universe_code_BPM,date,Name,Bloomberg Ticker,BloombergID,Asset ID,Asset ID Type,SEDOL,Country,...,FG_COMPANY_NAME_CUSIP,P_SYMBOL_CUSIP,ISIN,FG_COMPANY_NAME_ISIN,P_SYMBOL_ISIN,CODE_JP,FG_COMPANY_NAME_CODE_JP,P_SYMBOL_CODE_JP,P_SYMBOL,FG_COMPANY_NAME
400859,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZIMMER BIOMET HOLDINGS INC,None,None,USA4JT1,BARRAID,2783815,USA,...,ジンマー・バイオメット・ホールディングス,ZBH-US,US98956P1021,ジンマー・バイオメット・ホールディングス,ZBH-US,None,None,None,ZBH-US,ジンマー・バイオメット・ホールディングス
400860,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZOETIS INC,None,None,USBANZ1,BARRAID,B95WG16,USA,...,ゾエティス Class A,ZTS-US,US98978V1035,ゾエティス Class A,ZTS-US,None,None,None,ZTS-US,ゾエティス Class A
400861,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZOOM COMMUNICATIONS INC,None,None,USBEOV1,BARRAID,BGSP7M9,USA,...,ズーム・ビデオ・コミュニケーションズ Class A,ZM-US,US98980L1017,ズーム・ビデオ・コミュニケーションズ Class A,ZM-US,None,None,None,ZM-US,ズーム・ビデオ・コミュニケーションズ Class A
400862,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZSCALER INC,None,None,USBDYI1,BARRAID,BZ00V34,USA,...,ゼットスケイラー,ZS-US,US98980G1022,ゼットスケイラー,ZS-US,None,None,None,ZS-US,ゼットスケイラー
400863,MSCI KOKUSAI - Daily,MSXJPN_AD,2025-10-31,ZURICH INSURANCE GROUP AG,None,None,SWIAFM2,BARRAID,5983816,CHE,...,チューリッヒ・インシュアランス・グループ,ZFSVF-US,CH0011075394,チューリッヒ・インシュアランス・グループ,ZURN-CH,None,None,None,ZURN-CH,チューリッヒ・インシュアランス・グループ


In [ ]:
# 各変数を処理する関数
def process_variable(variable, financials_db_path, df_weight):
    with sqlite3.connect(financials_db_path) as conn:
        df_factor = pd.read_sql(
            f"SELECT `date`, `P_SYMBOL`, `value` FROM `{variable}`",
            con=conn,
            parse_dates=["date"],
        )

    merged_df = (
        pd.merge(df_weight, df_factor, on=["date", "P_SYMBOL"], how="outer")
        .rename(columns={"value": variable})
        .dropna(subset=["Weight (%)", variable], how="any", axis=0)
    ).fillna(np.nan)

    g = (
        pd.DataFrame(merged_df.groupby(["date"])["Weight (%)"].agg("sum"))
        .reset_index()
        .assign(variable=variable)
    )

    return g


In [ ]:
dfs_weight_sum = []
with ThreadPoolExecutor(max_workers=4) as executor:
    future_to_var = {
        executor.submit(process_variable, var, financials_db_path, df_weight): var
        for var in table_names
    }

    for future in as_completed(future_to_var):
        variable = future_to_var[future]
        try:
            result = future.result()
            if result is not None:
                dfs_weight_sum.append(result)
                print(f"✓ Completed: {variable}")
        except Exception as e:
            print(f"✗ Failed {variable}: {e}")


✓ Completed: Active_Return_3M
✓ Completed: Active_Return_1M_annlzd
✓ Completed: Active_Return_1M
✓ Completed: Active_Return_3M_annlzd
✓ Completed: Active_Return_3Y
✓ Completed: Active_Return_3Y_annlzd
✓ Completed: Active_Return_5Y
✓ Completed: Active_Return_5Y_annlzd
✓ Completed: Active_Return_6M
✓ Completed: FF_ASSETS
✓ Completed: Active_Return_6M_annlzd
✓ Completed: FF_ASSETS_CAGR_3Y
✓ Completed: FF_ASSETS_CAGR_5Y
✓ Completed: FF_ASSETS_CAGR_3Y_Rank
✓ Completed: FF_ASSETS_CAGR_3Y_PctRank
✓ Completed: FF_ASSETS_CAGR_3Y_ZScore
✓ Completed: FF_ASSETS_CAGR_5Y_PctRank
✓ Completed: FF_ASSETS_CAGR_5Y_Rank
✓ Completed: FF_ASSETS_CAGR_5Y_ZScore
✓ Completed: FF_ASSETS_PctRank
✓ Completed: FF_ASSETS_QoQ
✓ Completed: FF_ASSETS_QoQ_Rank
✓ Completed: FF_ASSETS_QoQ_PctRank
✓ Completed: FF_ASSETS_QoQ_ZScore
✓ Completed: FF_ASSETS_Rank
✓ Completed: FF_ASSETS_YoY
✓ Completed: FF_ASSETS_YoY_PctRank
✓ Completed: FF_ASSETS_YoY_Rank
✓ Completed: FF_ASSETS_YoY_ZScore
✓ Completed: FF_BPS
✓ Completed: FF_ASS

In [ ]:
df_weight_sum = pd.concat(dfs_weight_sum).sort_values(
    ["date", "Weight (%)"], ignore_index=True
)
df_weight_sum = (
    pd.pivot(df_weight_sum, index=["date"], columns="variable", values="Weight (%)")
    .reset_index()
    .filter(regex="date|_Rank|_PctRank|_ZScore")
)
display(df_weight_sum)

output_path = BLOOMBERG_DATA_DIR / f"{UNIVERSE_CODE}_not_missing_weight.xlsx"
df_weight_sum.to_excel(output_path, index=False)


variable,date,FF_ASSETS_CAGR_3Y_PctRank,FF_ASSETS_CAGR_3Y_Rank,FF_ASSETS_CAGR_3Y_ZScore,FF_ASSETS_CAGR_5Y_PctRank,FF_ASSETS_CAGR_5Y_Rank,FF_ASSETS_CAGR_5Y_ZScore,FF_ASSETS_PctRank,FF_ASSETS_QoQ_PctRank,FF_ASSETS_QoQ_Rank,...,FF_STK_OPT_EXP_ZScore,FF_STK_PURCH_CF_PctRank,FF_STK_PURCH_CF_Rank,FF_STK_PURCH_CF_ZScore,FF_TAX_RATE_PctRank,FF_TAX_RATE_Rank,FF_TAX_RATE_ZScore,FF_WKCAP_PctRank,FF_WKCAP_Rank,FF_WKCAP_ZScore
0,2005-08-31,NaN,NaN,NaN,NaN,NaN,NaN,97.532947,NaN,NaN,...,15.983133,61.725108,61.725108,59.932037,93.525348,93.525348,93.525348,89.346444,89.346444,89.346444
1,2005-09-30,NaN,NaN,NaN,NaN,NaN,NaN,96.534935,NaN,NaN,...,15.871030,62.047600,62.047600,51.564443,93.419152,93.419152,93.419152,89.205829,89.205829,89.205829
2,2005-10-31,NaN,NaN,NaN,NaN,NaN,NaN,96.471981,NaN,NaN,...,16.097034,61.961402,61.961402,52.128179,93.798649,93.798649,93.798649,89.420052,89.420052,89.420052
3,2005-11-30,NaN,NaN,NaN,NaN,NaN,NaN,96.472460,97.503718,97.503718,...,16.909461,62.512187,62.512187,52.846167,93.905481,93.905481,93.905481,89.080571,89.080571,89.080571
4,2005-12-31,NaN,NaN,NaN,NaN,NaN,NaN,97.655170,97.996633,97.996633,...,20.332567,62.126714,62.126714,60.209553,91.484261,91.484261,91.484261,93.541597,93.541597,93.541597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,2025-06-30,99.000611,99.000611,99.000611,97.892807,97.892807,97.892807,99.504106,99.999989,99.999989,...,77.992514,97.671326,97.671326,93.137308,97.224061,97.224061,97.224061,99.363345,99.363345,99.363345
239,2025-07-31,99.073587,99.073587,99.073587,97.926685,97.926685,97.926685,99.627776,100.000012,100.000012,...,78.723921,97.738047,97.738047,93.201061,97.311733,97.311733,97.311733,99.490787,99.490787,99.490787
240,2025-08-31,99.090801,99.090801,99.090801,97.910641,97.910641,97.910641,99.625314,100.000009,100.000009,...,78.425465,97.698057,97.698057,93.258100,97.260548,97.260548,97.260548,99.488512,99.488512,99.488512
241,2025-09-30,98.275030,98.275030,98.275030,97.371288,97.371288,97.371288,98.765681,99.999991,99.999991,...,77.311625,96.750014,96.750014,92.399361,97.000819,97.000819,97.000819,98.463638,98.463638,98.463638


In [ ]:
# 欠損確認（Weight (%)）
dfs_weight_sum = []
with sqlite3.connect(financials_db_path) as conn:
    for variable in table_names:
        df_factor = pd.read_sql(
            f"SELECT `date`, `P_SYMBOL`, `value` FROM `{variable}`",
            con=conn,
            parse_dates=["date"],
        )
        merged_df = (
            pd.merge(df_weight, df_factor, on=["date", "P_SYMBOL"], how="outer")
            .rename(columns={"value": variable})
            .dropna(subset=["Weight (%)", variable], how="any", axis=0)
        ).fillna(np.nan)

        g = (
            pd.DataFrame(merged_df.groupby(["date"])["Weight (%)"].agg("sum"))
            .reset_index()
            .assign(variable=variable)
        )

        dfs_weight_sum.append(g)

df_weight_sum = pd.concat(dfs_weight_sum, ignore_index=True)
df_weight_sum = pd.pivot(
    df_weight_sum, index=["date"], columns="variable", values="Weight (%)"
).reset_index()
display(df_weight_sum)


KeyboardInterrupt: 